In v3, higher level of filtering of high frequency words is used for constructing the vocab.  Low frequency filtering relaxed from min. 10 docs to 5 docs.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from collections import Counter
from six import iteritems
import warnings
import logging

import spacy
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore, LdaModel
from gensim.models.coherencemodel import CoherenceModel

import pyLDAvis.gensim

from tqdm._tqdm_notebook import tqdm, tqdm_notebook, tnrange
from S3_read_write import load_df_s3, save_df_s3

from IPython.display import Image
from IPython.core.display import HTML

In [4]:
tqdm_notebook.pandas('Progress')

In [5]:
bucket_name = 'amazon-reviews-project'

In [5]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.DEBUG)

# Load Amazon Reviews Data

Let us start off using only the title (`summary`) and body (`reviewText`) of each review.

In [5]:
%%time
df = load_df_s3(bucket_name, filepath='amazon_reviews/data_clean_v3', filetype='feather')

# df = load_df_s3(bucket_name, filepath='amazon_reviews/reviews_data_clean_v2.feather', filetype='feather')

CPU times: user 2.11 s, sys: 1.94 s, total: 4.05 s
Wall time: 10.7 s


In [6]:
df.dtypes

index                 int64
asin                 object
helpful              object
reviewText           object
overall             float64
summary              object
description          object
title                object
categories_clean     object
cat1                 object
cat2                 object
cat3                 object
cat4                 object
cat5                 object
cat6                 object
cat7                 object
dtype: object

In [7]:
df = df.loc[:, ['asin', 'reviewText', 'summary']]

In [8]:
df.shape

(217530, 3)

In [9]:
df.head()

,asin,reviewText,summary
0,0978559088,I started taking this after both my parents di...,Bioavailability is the key
1,0978559088,I really liked this product because it stayed ...,"I can't find this product any longer, and I wi..."
2,0978559088,"Resveratrol is a polar compound, very insolubl...",Just the Resveratrol product we need
3,1427600228,I bought several of these bracelets for my YMC...,The kids love these bracelets
4,1427600228,I bought a few the other week just to see what...,Pleasant Surprise


In [10]:
# for each review, concatenate the review title and body
df.reviewText = df.summary + '. ' + df.reviewText

In [11]:
pd.set_option('max_colwidth', 200)
df.head()

,asin,reviewText,summary
0,0978559088,Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspirin...,Bioavailability is the key
1,0978559088,"I can't find this product any longer, and I wish I could.. I really liked this product because it stayed in my mouth for a long time and I felt it was probably doing some good. I take a number of...","I can't find this product any longer, and I wish I could."
2,0978559088,"Just the Resveratrol product we need. Resveratrol is a polar compound, very insoluble in water and hence saliva. To get sufficient Resveratrol absorbed a Resveratrol lozenge would have to stay in...",Just the Resveratrol product we need
3,1427600228,"The kids love these bracelets. I bought several of these bracelets for my YMCA kids. Everyone tells me that it brought good luck. Placebo effect, perhaps but it's a positive effect and builds conf...",The kids love these bracelets
4,1427600228,Pleasant Surprise. I bought a few the other week just to see what they're all about. The first day I wore one of the bracelets three people asked about it. They liked the look. I told them it w...,Pleasant Surprise


Let's drop the `summary` column now:

In [12]:
df.drop(['summary'], axis=1, inplace=True)

In [13]:
df.head()

,asin,reviewText
0,0978559088,Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspirin...
1,0978559088,"I can't find this product any longer, and I wish I could.. I really liked this product because it stayed in my mouth for a long time and I felt it was probably doing some good. I take a number of..."
2,0978559088,"Just the Resveratrol product we need. Resveratrol is a polar compound, very insoluble in water and hence saliva. To get sufficient Resveratrol absorbed a Resveratrol lozenge would have to stay in..."
3,1427600228,"The kids love these bracelets. I bought several of these bracelets for my YMCA kids. Everyone tells me that it brought good luck. Placebo effect, perhaps but it's a positive effect and builds conf..."
4,1427600228,Pleasant Surprise. I bought a few the other week just to see what they're all about. The first day I wore one of the bracelets three people asked about it. They liked the look. I told them it w...


In [14]:
pd.set_option('max_colwidth', 20)

## Remove Missing Reviews

In [15]:
df.reviewText.isnull().sum()    # 73 reviews have neither a review body text, nor a review title

0

Let's look at a few actual review texts:

In [16]:
df.reviewText.iloc[np.random.randint(0, len(df))]

'Great product. A friend told me about this and I love it. It works just like it says. Highly recommend this product.'

In [17]:
df.reviewText.iloc[np.random.randint(0, len(df))]

'Yes! It works fast and lowers your BP withi a month.. It works...it works...lowered my blood pressure by 30 points. I take 2 pills in the AM and 2 in the afternoon. My BP was high and now it is borderline.'

In [18]:
df.reviewText.iloc[np.random.randint(0, len(df))]

"I love it!. I am very sensitive to most things. I can't drink coffee, I can't drink soda, I can't drink most green teas. However with this, I can have just one teaspoon of the Matcha Green Tea with water and I feel a gradual alertness that stays for quite some time with no crashing. I also find it curbs my appetite. Love it!"

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217530 entries, 0 to 217529
Data columns (total 2 columns):
asin          217530 non-null object
reviewText    217530 non-null object
dtypes: object(2)
memory usage: 3.3+ MB


## Phrase Detection

In [20]:
text = list(df.reviewText.values)    # make an iterable to store only the review text

In [21]:
[sent for sent in text if len(sent) == 0]   # there are no blank sentences

[]

In [22]:
len(text)

217530

In [23]:
# look at a few sample reviews
for rev in text[:4]:
    print(rev, '\n')

Bioavailability is the key. I started taking this after both my parents died of cancer as it supposed to enhance your immune system - the story on 60 Minutes on resveratrol was incredibly inspiring. Doing some research on the Internet, it is indicated that taking resveratrol in lozenge form is preferable as it is broken down by stomach acids.  The ez-melt formula recommended in another review is OK, but it is dissolved in the mouth much more quickly than this lozenge formula, while dissolving more slowly is preferable according to my research.This product has the greatest side effect - since taking it, I haven't had colds or sore throats.  Soon after starting to take it every day, I was starting to come down with a cold, with all my usual symptoms, and was anticipating being very sick the next day, as is my usual pattern.  But I never did get as sick as anticipated - taking this product is the only reason I can come up with.  Since then, I've had no colds or sore throats - it has been 

In [24]:
nlp = spacy.load('en')

The helper functions below are from:

http://nbviewer.jupyter.org/github/skipgram/modern-nlp-in-python/blob/master/executable/Modern_NLP_in_Python.ipynb

In order to use `gensim`'s `Phrases` class to detect natural combinations of words (like 'vanilla ice cream'), we need to format our text into a list of sentences, with each sentence being a list of words.  This process takes a large amount of processing time (for reference, the times shown under the cells are for running the tasks on a c5.18xlarge EC2 instance (equivalent spot fleet)), so `text` has been split into 3 parts.

### Generate Unigram Sentences

In [25]:
len(text)

217530

In [26]:
# split text into 9 parts
text_first  = text[:50000]
text_second = text[50000:100000]
text_third  = text[100000:150000]
text_fourth = text[150000:]

In [27]:
rev_num = 0    # review tracker
sent_num = 0   # sentence tracker
unigram_sents_pos = [] # to store lists of lemmatized tokens for each sentence

for parsed_review in tqdm(nlp.pipe(text_first, batch_size=10000, n_threads=72)):
    rev_num += 1
    for sent in parsed_review.sents:
        sent_num += 1
        # lemmatize tokens & save corresponding pos tag after filtering whitespace and punctuations
        lemmatized_sent = [(token.lemma_, token.pos_) for token in sent if not (token.is_space or token.is_punct)]
        if len(lemmatized_sent) != 0:
            unigram_sents_pos.append([rev_num, sent_num, lemmatized_sent])

print('current rev_num: ', rev_num)
print('current sent_num: ', sent_num)

50000it [06:56, 120.02it/s]

current rev_num:  50000
current sent_num:  289565


In [28]:
len(unigram_sents_pos)

289158

In [29]:
for i in range(5):
    print(unigram_sents_pos[i])

[1, 1, [('bioavailability', 'PROPN'), ('be', 'VERB'), ('the', 'DET'), ('key', 'NOUN')]]
[1, 2, [('-PRON-', 'PRON'), ('start', 'VERB'), ('take', 'VERB'), ('this', 'DET'), ('after', 'ADP'), ('both', 'CCONJ'), ('-PRON-', 'ADJ'), ('parent', 'NOUN'), ('die', 'VERB'), ('of', 'ADP'), ('cancer', 'NOUN'), ('as', 'ADP'), ('-PRON-', 'PRON'), ('suppose', 'VERB'), ('to', 'PART'), ('enhance', 'VERB'), ('-PRON-', 'ADJ'), ('immune', 'ADJ'), ('system', 'NOUN'), ('the', 'DET'), ('story', 'NOUN'), ('on', 'ADP'), ('60', 'NUM'), ('minutes', 'PROPN'), ('on', 'ADP'), ('resveratrol', 'NOUN'), ('be', 'VERB'), ('incredibly', 'ADV'), ('inspiring', 'ADJ')]]
[1, 3, [('do', 'VERB'), ('some', 'DET'), ('research', 'NOUN'), ('on', 'ADP'), ('the', 'DET'), ('internet', 'NOUN'), ('-PRON-', 'PRON'), ('be', 'VERB'), ('indicate', 'VERB'), ('that', 'ADP'), ('take', 'VERB'), ('resveratrol', 'NOUN'), ('in', 'ADP'), ('lozenge', 'NOUN'), ('form', 'NOUN'), ('be', 'VERB'), ('preferable', 'ADJ'), ('as', 'ADP'), ('-PRON-', 'PRON'), 

In [30]:
# check if there are any blank sentences
for sent in unigram_sents_pos:
    if len(sent[2]) == 0:
        print(sent)

In [31]:
# Save progress...
review_number = [row[0] for row in unigram_sents_pos]
sentence_number = [row[1] for row in unigram_sents_pos]
words_joined_all = []
pos_joined_all = []
for sent in unigram_sents_pos:
    word_pos = sent[2]
    word_list = [word for word, pos in word_pos]
    pos_list = [pos for word, pos in word_pos]
    words_joined = '+-+||+-+'.join(word for word in word_list)
    pos_joined   = '+-+||+-+'.join(pos for pos in pos_list)
    words_joined_all.append(words_joined)
    pos_joined_all.append(pos_joined)
    
unigram_sentences_savedf = pd.DataFrame({'review_number': review_number,
                                         'sentence_number': sentence_number,
                                         'unigram_sentences': words_joined_all,
                                         'unigram_pos': pos_joined_all})

save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/kk/unigram_sentences.feather')

In [32]:
for parsed_review in tqdm(nlp.pipe(text_second, batch_size=20000, n_threads=36)):
    rev_num += 1
    for sent in parsed_review.sents:
        sent_num += 1
        # lemmatize tokens & save corresponding pos tag after filtering whitespace and punctuations
        lemmatized_sent = [(token.lemma_, token.pos_) for token in sent if not (token.is_space or token.is_punct)]
        if len(lemmatized_sent) != 0:
            unigram_sents_pos.append([rev_num, sent_num, lemmatized_sent])

print('current rev_num: ', rev_num)
print('current sent_num: ', sent_num)

50000it [06:55, 120.43it/s]

current rev_num:  100000
current sent_num:  576892


In [33]:
print(len(unigram_sents_pos))

576034


In [34]:
# Save progress...
review_number = [row[0] for row in unigram_sents_pos]
sentence_number = [row[1] for row in unigram_sents_pos]
words_joined_all = []
pos_joined_all = []
for sent in unigram_sents_pos:
    word_pos = sent[2]
    word_list = [word for word, pos in word_pos]
    pos_list = [pos for word, pos in word_pos]
    words_joined = '+-+||+-+'.join(word for word in word_list)
    pos_joined   = '+-+||+-+'.join(pos for pos in pos_list)
    words_joined_all.append(words_joined)
    pos_joined_all.append(pos_joined)
    
unigram_sentences_savedf = pd.DataFrame({'review_number': review_number,
                                         'sentence_number': sentence_number,
                                         'unigram_sentences': words_joined_all,
                                         'unigram_pos': pos_joined_all})

save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/kk/unigram_sentences.feather')

In [35]:
for parsed_review in tqdm(nlp.pipe(text_third, batch_size=20000, n_threads=36)):
    rev_num += 1
    for sent in parsed_review.sents:
        sent_num += 1
        # lemmatize tokens & save corresponding pos tag after filtering whitespace and punctuations
        lemmatized_sent = [(token.lemma_, token.pos_) for token in sent if not (token.is_space or token.is_punct)]
        if len(lemmatized_sent) != 0:
            unigram_sents_pos.append([rev_num, sent_num, lemmatized_sent])

print('current rev_num: ', rev_num)
print('current sent_num: ', sent_num)

50000it [06:27, 129.16it/s]

current rev_num:  150000
current sent_num:  855567


In [36]:
# Save progress...
review_number = [row[0] for row in unigram_sents_pos]
sentence_number = [row[1] for row in unigram_sents_pos]
words_joined_all = []
pos_joined_all = []
for sent in unigram_sents_pos:
    word_pos = sent[2]
    word_list = [word for word, pos in word_pos]
    pos_list = [pos for word, pos in word_pos]
    words_joined = '+-+||+-+'.join(word for word in word_list)
    pos_joined   = '+-+||+-+'.join(pos for pos in pos_list)
    words_joined_all.append(words_joined)
    pos_joined_all.append(pos_joined)
    
unigram_sentences_savedf = pd.DataFrame({'review_number': review_number,
                                         'sentence_number': sentence_number,
                                         'unigram_sentences': words_joined_all,
                                         'unigram_pos': pos_joined_all})

save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/kk/unigram_sentences.feather')

In [37]:
for parsed_review in tqdm(nlp.pipe(text_fourth, batch_size=20000, n_threads=36)):
    rev_num += 1
    for sent in parsed_review.sents:
        sent_num += 1
        # lemmatize tokens & save corresponding pos tag after filtering whitespace and punctuations
        lemmatized_sent = [(token.lemma_, token.pos_) for token in sent if not (token.is_space or token.is_punct)]
        if len(lemmatized_sent) != 0:
            unigram_sents_pos.append([rev_num, sent_num, lemmatized_sent])

print('current rev_num: ', rev_num)
print('current sent_num: ', sent_num)

67530it [09:13, 122.02it/s]

current rev_num:  217530
current sent_num:  1243596


In [38]:
# Save progress...
review_number = [row[0] for row in unigram_sents_pos]
sentence_number = [row[1] for row in unigram_sents_pos]
words_joined_all = []
pos_joined_all = []
for sent in unigram_sents_pos:
    word_pos = sent[2]
    word_list = [word for word, pos in word_pos]
    pos_list = [pos for word, pos in word_pos]
    words_joined = '+-+||+-+'.join(word for word in word_list)
    pos_joined   = '+-+||+-+'.join(pos for pos in pos_list)
    words_joined_all.append(words_joined)
    pos_joined_all.append(pos_joined)
    
unigram_sentences_savedf = pd.DataFrame({'review_number': review_number,
                                         'sentence_number': sentence_number,
                                         'unigram_sentences': words_joined_all,
                                         'unigram_pos': pos_joined_all})

save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/kk/unigram_sentences.feather')

In [39]:
# DON'T LOAD THIS FILE - there's a _v1 version further down!
# del unigram_sentences_savedf
unigram_sentences_savedf = load_df_s3(bucket_name, 'amazon_reviews/kk/unigram_sentences.feather', filetype='feather')

In [40]:
unigram_sentences_savedf.head()

,review_number,sentence_number,unigram_pos,unigram_sentences
0,1,1,PROPN+-+||+-+VER...,bioavailability+...
1,1,2,PRON+-+||+-+VERB...,-PRON-+-+||+-+st...
2,1,3,VERB+-+||+-+DET+...,do+-+||+-+some+-...
3,1,4,DET+-+||+-+ADP+-...,the+-+||+-+ez+-+...
4,1,5,DET+-+||+-+NOUN+...,this+-+||+-+prod...


In [41]:
unigram_sentences_savedf[unigram_sentences_savedf.unigram_pos == ''].shape

(0, 4)

In [42]:
unigram_sentences_savedf[unigram_sentences_savedf.unigram_sentences == ''].shape

(0, 4)

In [43]:
unigram_sentences_savedf[unigram_sentences_savedf.unigram_pos == ''].head()  # no blank sentences

,review_number,sentence_number,unigram_pos,unigram_sentences


In [44]:
unigram_sentences_savedf[unigram_sentences_savedf.unigram_pos == ''].shape

(0, 4)

#### Additional Data Cleaning

In [45]:
def clean_up(sentence, sentence_pos, sep):
    """Expects a sentence as a single string as input 1, and its corresponding part-of-speech tags as input 2 (also single string).
    sep is the string pattern used to separate words in each sentence string
    Cleans it up and returns a single string.
    Also updates corresponding part-of-speech string.
    """
    # get rid of webpage links
    cond = ['http' in sentence, 'www' in sentence]
    if any(cond):
        words = sentence.split(sep)
        words_pos = sentence_pos.split(sep)
        to_remove = []
        for i in range(len(words)):
            cond_word = ['http' in words[i], 'www' in words[i]]
            if any(cond_word):
                to_remove.append(i)
        # remove words that are links
        for j in sorted(to_remove, reverse=True):
            del words[j]
            del words_pos[j]
        # reconstruct sentence after deleting links
        sentence = sep.join(words)
        sentence_pos = sep.join(words_pos)

    # replace underscores with blanks to avoid mix-up with paired words later
    # cannot replace with spaces because the strings are split on spaces later 
    # and this would create new words with no corresponding pos tags
    if '_' in sentence:
        sentence = sentence.replace('_', '')
    return sentence, sentence_pos

In [46]:
test_clean = ['whoa watch out for them links boy http://sup.com and also BAM! underscore_time!', 'this is a normal sentence', 
              '__ what is this ____ http', '_', 'http']
test_clean

['whoa watch out for them links boy http://sup.com and also BAM! underscore_time!',
 'this is a normal sentence',
 '__ what is this ____ http',
 '_',
 'http']

In [47]:
test_clean_pos = ['X X X X X X X X X X X X', 'X X X X X', 'X X X X X X', 'X', 'X']

In [48]:
[len(e.split(' ')) for e in test_clean]

[12, 5, 6, 1, 1]

In [49]:
[e.count('X') for e in test_clean_pos]

[12, 5, 6, 1, 1]

In [50]:
# check if clean_up works as expected
to_remove = []
for i in range(len(test_clean)):
    sentence = test_clean[i]
    sentence_pos = test_clean_pos[i]
    test_clean[i], test_clean_pos[i] = clean_up(sentence, sentence_pos, sep=' ')
    
    # mark elements to delete if empty
    if test_clean[i] == '':
        to_remove.append(i)

# delete elements that are empty
for j in sorted(to_remove, reverse=True):
    del test_clean[j]
    del test_clean_pos[j]

test_clean

['whoa watch out for them links boy and also BAM! underscoretime!',
 'this is a normal sentence',
 ' what is this ']

In [51]:
test_clean_pos

['X X X X X X X X X X X', 'X X X X X', 'X X X X X']

In [52]:
[e.count('X') for e in test_clean_pos]

[11, 5, 5]

In [53]:
[len(e.split(' ')) for e in test_clean]

[11, 5, 5]

In [54]:
words_joined_all = unigram_sentences_savedf.unigram_sentences.tolist()

In [55]:
pos_joined_all = unigram_sentences_savedf.unigram_pos.tolist()

In [56]:
len(words_joined_all)

1241850

In [57]:
len([sentence for sentence in words_joined_all if '_' in sentence])

172

In [58]:
len([sentence for sentence in words_joined_all if 'http' in sentence])

126

In [59]:
len([sentence for sentence in words_joined_all if 'www' in sentence])

157

In [60]:
unigram_sentences_savedf[unigram_sentences_savedf.unigram_sentences.str.contains('_')].head()

,review_number,sentence_number,unigram_pos,unigram_sentences
3375,607,3386,X,http://www.amazo...
8051,1454,8071,DET+-+||+-+NOUN+...,no+-+||+-+jet_la...
12199,2166,12224,ADJ+-+||+-+PART+...,easy+-+||+-+to+-...
12201,2166,12226,PRON+-+||+-+VERB...,-PRON-+-+||+-+ha...
18188,3106,18214,ADV+-+||+-+ADJ+-...,overall+-+||+-+-...


In [61]:
[sentence for sentence in words_joined_all if '_' in sentence][:10]

['http://www.amazon.com/gp/product/b0000533z8/ref=cm_cr_rev_prod_title',
 'no+-+||+-+jet_lag+-+||+-+pill',
 'easy+-+||+-+to+-+||+-+use_work+-+||+-+well',
 '-PRON-+-+||+-+have+-+||+-+have+-+||+-+pedometer+-+||+-+in+-+||+-+the+-+||+-+past_all+-+||+-+difficult+-+||+-+and+-+||+-+confusing+-+||+-+to+-+||+-+use+-+||+-+to+-+||+-+the+-+||+-+point+-+||+-+-PRON-+-+||+-+simply+-+||+-+give+-+||+-+up+-+||+-+on+-+||+-+-PRON-',
 'overall+-+||+-+-PRON-+-+||+-+mother+-+||+-+be+-+||+-+very+-+||+-+satisfied+-+||+-+with+-+||+-+this+-+||+-+product!-d_lionz',
 'this+-+||+-+inexpensive+-+||+-+strap+-+||+-+with+-+||+-+a+-+||+-+metal+-+||+-+clip+-+||+-+http://www.amazon.com/gp/product/b000bitymg/ref=oh_details_o00_s00_i00?ie=utf8&psc;=1+-+||+-+be+-+||+-+a+-+||+-+good+-+||+-+replacement+-+||+-+for+-+||+-+the+-+||+-+flimsy+-+||+-+omron+-+||+-+plastic+-+||+-+clip+-+||+-+but+-+||+-+-PRON-+-+||+-+have+-+||+-+not+-+||+-+be+-+||+-+use+-+||+-+-PRON-+-+||+-+long',
 'hj_112+-+||+-+digital+-+||+-+pemium+-+||+-+pedometer+

In [62]:
# clean up all unigrams
to_remove = []
for i in range(len(words_joined_all)):
    sentence = words_joined_all[i]
    sentence_pos = pos_joined_all[i]
    words_joined_all[i], pos_joined_all[i] = clean_up(sentence, sentence_pos, sep='+-+||+-+')
    
    # mark elements to delete if empty
    if words_joined_all[i] == '':
        to_remove.append(i)

# delete elements that are empty
for j in sorted(to_remove, reverse=True):
    del words_joined_all[j]
    del pos_joined_all[j]

In [63]:
# drop rows from unigram_sentences_savedf corresponding to the row numbers (indices) of sentences
# that will be blank after the transformation above
unigram_sentences_savedf.drop(unigram_sentences_savedf.index[to_remove], axis=0, inplace=True)

In [64]:
unigram_sentences_savedf.drop(['unigram_sentences'], axis=1, inplace=True)
unigram_sentences_savedf.drop(['unigram_pos'], axis=1, inplace=True)

In [65]:
unigram_sentences_savedf['unigram_sentences'] = words_joined_all
unigram_sentences_savedf['unigram_pos'] = pos_joined_all

In [66]:
unigram_sentences_savedf.head()

,review_number,sentence_number,unigram_sentences,unigram_pos
0,1,1,bioavailability+...,PROPN+-+||+-+VER...
1,1,2,-PRON-+-+||+-+st...,PRON+-+||+-+VERB...
2,1,3,do+-+||+-+some+-...,VERB+-+||+-+DET+...
3,1,4,the+-+||+-+ez+-+...,DET+-+||+-+ADP+-...
4,1,5,this+-+||+-+prod...,DET+-+||+-+NOUN+...


In [68]:
unigram_sentences_savedf.shape

(1241826, 4)

In [69]:
# updated, cleaned up version of unigram_sentences.feather
save_df_s3(unigram_sentences_savedf, bucket_name, 'amazon_reviews/kk/unigram_sentences_v1.feather')

### Phrase Detection

In [21]:
unigram_sentences_savedf = load_df_s3(bucket_name, 'amazon_reviews/kk/unigram_sentences_v1.feather', filetype='feather')

In [70]:
words_joined_all = unigram_sentences_savedf.unigram_sentences.tolist()

In [71]:
unigram_sentences = [sentence.split('+-+||+-+') for sentence in words_joined_all]

In [72]:
print(unigram_sentences[:4])

[['bioavailability', 'be', 'the', 'key'], ['-PRON-', 'start', 'take', 'this', 'after', 'both', '-PRON-', 'parent', 'die', 'of', 'cancer', 'as', '-PRON-', 'suppose', 'to', 'enhance', '-PRON-', 'immune', 'system', 'the', 'story', 'on', '60', 'minutes', 'on', 'resveratrol', 'be', 'incredibly', 'inspiring'], ['do', 'some', 'research', 'on', 'the', 'internet', '-PRON-', 'be', 'indicate', 'that', 'take', 'resveratrol', 'in', 'lozenge', 'form', 'be', 'preferable', 'as', '-PRON-', 'be', 'break', 'down', 'by', 'stomach', 'acid'], ['the', 'ez', 'melt', 'formula', 'recommend', 'in', 'another', 'review', 'be', 'ok', 'but', '-PRON-', 'be', 'dissolve', 'in', 'the', 'mouth', 'much', 'more', 'quickly', 'than', 'this', 'lozenge', 'formula', 'while', 'dissolve', 'more', 'slowly', 'be', 'preferable', 'accord', 'to', '-PRON-', 'research']]


In [73]:
len(words_joined_all)

1241826

In [74]:
%%time
# The common_terms parameter add a way to give special treatment to common terms 
# (aka stop words) such that their presence between two words won’t prevent bigram detection. 
# It allows to detect expressions like “bank of america”
common_terms = ["of", "with", "without", "and", "or"]

# Train a first-order phrase detector
bigram_model = Phrases(unigram_sentences, threshold=0.6, scoring='npmi', common_terms=common_terms)

# Transform unigram sentences into bigram sentences
# Paired words are connected by an underscore, e.g. ice_cream
bigram_sentences = []
for sentence in unigram_sentences:
    bigram_sentences.append(bigram_model[sentence])

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1min 24s, sys: 1.12 s, total: 1min 25s
Wall time: 1min 25s


In [75]:
%%time
# Train a second-order phrase detector
# trigram_model = Phrases(bigram_sentences, min_count=5)
trigram_model = Phrases(bigram_sentences, threshold=0.5, scoring='npmi')

# Transform bigram sentences into trigram sentences
trigram_sentences = []
for sentence in bigram_sentences:
    trigram_sentences.append(trigram_model[sentence])

# remove any remaining stopwords
# trigram_sentences = [[word for word in sentence if word not in nlp.Defaults.stop_words] for sentence in trigram_sentences]

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


CPU times: user 1min 23s, sys: 1.29 s, total: 1min 24s
Wall time: 1min 24s


In [76]:
# the trigrams will be saved in a dataframe with a single column.
# each row is one sentence from any review
# each sentence is a single string separated by a single space.
trigram_sentences_savedf = pd.DataFrame([u'+-+||+-+'.join(sentence) for sentence in trigram_sentences], columns=['preprocessed_review'])
save_df_s3(trigram_sentences_savedf, bucket_name, 'amazon_reviews/kk/preprocessed_reviews.feather')

In [5]:
trigram_sentences_savedf = load_df_s3(bucket_name, 'amazon_reviews/kk/preprocessed_reviews.feather', filetype='feather')

In [77]:
trigram_sentences_savedf.head()

,preprocessed_review
0,bioavailability+...
1,-PRON-+-+||+-+st...
2,do+-+||+-+some+-...
3,the+-+||+-+ez+-+...
4,this_product+-+|...


In [78]:
del unigram_sentences_savedf
unigram_sents_pos_df = load_df_s3(bucket_name, 'amazon_reviews/kk/unigram_sentences_v1.feather', filetype='feather')

In [79]:
unigram_sents_pos_df.head()

,review_number,sentence_number,unigram_sentences,unigram_pos
0,1,1,bioavailability+...,PROPN+-+||+-+VER...
1,1,2,-PRON-+-+||+-+st...,PRON+-+||+-+VERB...
2,1,3,do+-+||+-+some+-...,VERB+-+||+-+DET+...
3,1,4,the+-+||+-+ez+-+...,DET+-+||+-+ADP+-...
4,1,5,this+-+||+-+prod...,DET+-+||+-+NOUN+...


In [80]:
unigram_sents_pos_df[unigram_sents_pos_df.unigram_pos == ''].shape

(0, 4)

In [81]:
unigram_sents_pos_df.shape

(1241826, 4)

In [82]:
unigram_sents_pos_df = pd.merge(unigram_sents_pos_df, trigram_sentences_savedf, how='inner', left_index=True, right_index=True)

In [83]:
unigram_sents_pos_df.head(10)

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review
0,1,1,bioavailability+...,PROPN+-+||+-+VER...,bioavailability+...
1,1,2,-PRON-+-+||+-+st...,PRON+-+||+-+VERB...,-PRON-+-+||+-+st...
2,1,3,do+-+||+-+some+-...,VERB+-+||+-+DET+...,do+-+||+-+some+-...
3,1,4,the+-+||+-+ez+-+...,DET+-+||+-+ADP+-...,the+-+||+-+ez+-+...
4,1,5,this+-+||+-+prod...,DET+-+||+-+NOUN+...,this_product+-+|...
5,1,6,soon+-+||+-+afte...,ADV+-+||+-+ADP+-...,soon+-+||+-+afte...
6,1,7,but+-+||+-+-PRON...,CCONJ+-+||+-+PRO...,but+-+||+-+-PRON...
7,1,8,since+-+||+-+the...,ADP+-+||+-+ADV+-...,since+-+||+-+the...
8,1,9,-PRON-+-+||+-+re...,PRON+-+||+-+VERB...,-PRON-+-+||+-+re...
9,2,10,-PRON-+-+||+-+ca...,PRON+-+||+-+VERB...,-PRON-+-+||+-+ca...


In [84]:
save_df_s3(unigram_sents_pos_df, bucket_name, 'amazon_reviews/kk/preprocessed_reviews_v1.feather')

In [113]:
unigram_sents_pos_df = load_df_s3(bucket_name, 'amazon_reviews/kk/preprocessed_reviews_v1.feather', filetype='feather')

In [85]:
unigram_sents_pos_df.shape

(1241826, 5)

In [86]:
unigram_sents_pos_df.iloc[400:410]

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review
400,73,401,-PRON-+-+||+-+ha...,PRON+-+||+-+VERB...,-PRON-+-+||+-+ha...
401,73,402,so+-+||+-+just+-...,ADV+-+||+-+ADV+-...,so+-+||+-+just+-...
402,73,403,-PRON-+-+||+-+ca...,PRON+-+||+-+VERB...,-PRON-+-+||+-+ca...
403,73,404,-PRON-+-+||+-+do...,PRON+-+||+-+VERB...,-PRON-+-+||+-+do...
404,73,405,but+-+||+-+-PRON...,CCONJ+-+||+-+PRO...,but+-+||+-+-PRON...
405,74,406,excellent+-+||+-...,INTJ+-+||+-+PRON...,excellent+-+||+-...
406,74,407,-PRON-+-+||+-+ha...,PRON+-+||+-+VERB...,-PRON-+-+||+-+ha...
407,74,408,-PRON-+-+||+-+ac...,PRON+-+||+-+ADV+...,-PRON-+-+||+-+ac...
408,74,409,-PRON-+-+||+-+be...,PRON+-+||+-+VERB...,-PRON-+-+||+-+be...
409,74,410,one+-+||+-+pill+...,NUM+-+||+-+NOUN+...,one+-+||+-+pill+...


In [87]:
unigram_sents_pos_df.isnull().sum()

review_number          0
sentence_number        0
unigram_sentences      0
unigram_pos            0
preprocessed_review    0
dtype: int64

In [88]:
unigram_sents_pos_df['has_paired_words'] = 0

In [89]:
unigram_sents_pos_df.loc[unigram_sents_pos_df.preprocessed_review.str.contains('_'), ['has_paired_words']] = 1

In [90]:
unigram_sents_pos_df.has_paired_words.sum()  # number of sentences with paired words

532356

In [91]:
unigram_sents_pos_df.head()

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review,has_paired_words
0,1,1,bioavailability+...,PROPN+-+||+-+VER...,bioavailability+...,0
1,1,2,-PRON-+-+||+-+st...,PRON+-+||+-+VERB...,-PRON-+-+||+-+st...,1
2,1,3,do+-+||+-+some+-...,VERB+-+||+-+DET+...,do+-+||+-+some+-...,1
3,1,4,the+-+||+-+ez+-+...,DET+-+||+-+ADP+-...,the+-+||+-+ez+-+...,0
4,1,5,this+-+||+-+prod...,DET+-+||+-+NOUN+...,this_product+-+|...,1


In [92]:
%%time
unigram_sents_pos_df.unigram_pos = [sent.split('+-+||+-+') for sent in unigram_sents_pos_df.unigram_pos.tolist()]
unigram_sents_pos_df.unigram_sentences = [sent.split('+-+||+-+') for sent in unigram_sents_pos_df.unigram_sentences.tolist()]
unigram_sents_pos_df.preprocessed_review = [sent.split('+-+||+-+') for sent in unigram_sents_pos_df.preprocessed_review.tolist()]

CPU times: user 13.5 s, sys: 2.31 s, total: 15.8 s
Wall time: 15.8 s


In [93]:
unigram_sents_pos_df.iloc[400:410]

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review,has_paired_words
400,73,401,"[-PRON-, have, u...","[PRON, VERB, VER...","[-PRON-, have, u...",1
401,73,402,"[so, just, take,...","[ADV, ADV, VERB,...","[so, just, take,...",0
402,73,403,"[-PRON-, can, no...","[PRON, VERB, ADV...","[-PRON-, can_not...",1
403,73,404,"[-PRON-, do, not...","[PRON, VERB, ADV...","[-PRON-, do_not,...",1
404,73,405,"[but, -PRON-, be...","[CCONJ, PRON, VE...","[but, -PRON-, be...",0
405,74,406,"[excellent, prod...","[INTJ, PRON, ADJ...","[excellent, prod...",0
406,74,407,"[-PRON-, have, t...","[PRON, VERB, VER...","[-PRON-, have, t...",1
407,74,408,"[-PRON-, actuall...","[PRON, ADV, VERB...","[-PRON-, actuall...",1
408,74,409,"[-PRON-, be, con...","[PRON, VERB, ADJ...","[-PRON-, be, con...",0
409,74,410,"[one, pill, seem...","[NUM, NOUN, VERB...","[one, pill, seem...",1


In [94]:
unigram_sents_pos_df.isnull().sum()

review_number          0
sentence_number        0
unigram_sentences      0
unigram_pos            0
preprocessed_review    0
has_paired_words       0
dtype: int64

Let's look at an arbitrary sentence and it's transformation:

In [95]:
print(unigram_sents_pos_df.unigram_sentences.iloc[105])

['-PRON-', 'have', 'buy', 'nu', 'skin', 'product', 'about', '5', 'year', 'ago', 'from', 'a', 'beauty', 'salon', 'and', '-PRON-', 'love', '-PRON-']


In [96]:
print(unigram_sents_pos_df.unigram_pos.iloc[105])

['PRON', 'VERB', 'VERB', 'PROPN', 'PROPN', 'NOUN', 'ADV', 'NUM', 'NOUN', 'ADV', 'ADP', 'DET', 'NOUN', 'NOUN', 'CCONJ', 'PRON', 'VERB', 'PRON']


In [97]:
print(unigram_sents_pos_df.preprocessed_review.iloc[105])

['-PRON-', 'have', 'buy', 'nu', 'skin', 'product', 'about', '5', 'year_ago', 'from', 'a', 'beauty', 'salon', 'and', '-PRON-', 'love', '-PRON-']


In [98]:
gramlist = [word for sent in trigram_sentences for word in sent if '_' in word]

In [99]:
paired_words_frq = Counter(gramlist)
paired_words_frq.most_common(100)

[('this_product', 69494),
 ('do_not', 59653),
 ('seem_to', 14671),
 ('can_not', 14216),
 ('fish_oil', 12919),
 ('highly_recommend', 12415),
 ('as_well', 10388),
 ('lot_of', 9337),
 ('so_far', 6983),
 ('would_recommend', 6173),
 ('will_continue', 5383),
 ('every_day', 5290),
 ('side_effect', 5008),
 ('along_with', 4968),
 ('vitamin_d', 4601),
 ('at_least', 4305),
 ('per_day', 4262),
 ('high_quality', 4237),
 ('year_ago', 4147),
 ('at_night', 3881),
 ('very_happy', 3827),
 ('vitamin_c', 3800),
 ('suffer_from', 3715),
 ('immune_system', 3494),
 ('five_star', 3471),
 ('omega_3', 3426),
 ('run_out', 3420),
 ('year_old', 3134),
 ('long_time', 3126),
 ('no_longer', 3036),
 ('krill_oil', 3001),
 ('make_sure', 2684),
 ('out_there', 2672),
 ('very_pleased', 2620),
 ('wake_up', 2582),
 ('blood_pressure', 2492),
 ('energy_level', 2438),
 ('no_side_effect', 2353),
 ('get_sick', 2260),
 ('anyone_who', 2208),
 ('every_morning', 2101),
 ('go_away', 2018),
 ('joint_pain', 1996),
 ('weight_loss', 1992),

In [100]:
# Find the 100 most infrequent paired words
paired_words_frq.most_common()[::-1][:100]

[('wishful_thinking!ftc_disclosure', 1),
 ('none_after!.', 1),
 ('34;healthy_fat&#34', 1),
 ('atrail_fibrillationso', 1),
 ('superior_product!paula', 1),
 ('ever!!!highly_recommended', 1),
 ('george_flansbaum_whom', 1),
 ('side_effects).this', 1),
 ('side_effects!so', 1),
 ('onelife_pharma_sound', 1),
 ('coco_mak_seriously', 1),
 ('34;off_days&#34', 1),
 ('i&#8217;m_assuming', 1),
 ('yeast_infection_every2', 1),
 ('plant_derived', 1),
 ('protease_enzym', 1),
 ('virtually_untreatable', 1),
 ('con_su_rodilla', 1),
 ('mi_mama', 1),
 ('ayuda_mucho', 1),
 ('productmuy_buen_producto', 1),
 ('sodium_free.&#34_wow', 1),
 ('1000_mg_serving!.', 1),
 ('countless_cortizone', 1),
 ('tons_of_benefits!.', 1),
 ('34;clear_out&#34', 1),
 ('bri_nutrition&#8217;s_unconditional_guarantee', 1),
 ('dieting_pilling', 1),
 ('nutrition!._bri_nutrition', 1),
 ('garage_and_repaint', 1),
 ('bri_nutrition_triphalia', 1),
 ('unhygienic_colon', 1),
 ('slouchy_and_drain', 1),
 ('8220;bowel_issues&#8221_lately', 1),
 

In [101]:
len(paired_words_frq)  # number of paired terms  (this drops down to 46,785 after further processing)

83377

In [102]:
unigram_sents_pos_df.head()

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review,has_paired_words
0,1,1,[bioavailability...,"[PROPN, VERB, DE...",[bioavailability...,0
1,1,2,"[-PRON-, start, ...","[PRON, VERB, VER...","[-PRON-, start, ...",1
2,1,3,"[do, some, resea...","[VERB, DET, NOUN...","[do, some, resea...",1
3,1,4,"[the, ez, melt, ...","[DET, ADP, NOUN,...","[the, ez, melt, ...",0
4,1,5,"[this, product, ...","[DET, NOUN, VERB...","[this_product, h...",1


In [103]:
def handle_failed_pairing(i, skip, num_paired, sent, sent_paired, to_remove):
    # split up paired words failing our format requirements
    to_remove.extend([i])
    sent_paired.extend(sent[i + skip: i + skip + num_paired])


def filter_pairs(sent, sent_paired, sent_pos):
    """modify sent_paired in place"""
    paired_sent_len = len(sent_paired)
    skip = 0
    to_remove = []
    
    for i in range(paired_sent_len):
        word = sent_paired[i]
        if '_' in word:
            num_paired = word.count('_') + 1
            
            # more than 3 words paired - ignore pairing
            if num_paired > 3:
                handle_failed_pairing(i, skip, num_paired, sent, sent_paired, to_remove)
                skip += num_paired - 1
                continue
            
            # bigrams: noun/adj, noun
            elif num_paired == 2:
                pos_word_1 = sent_pos[i + skip]
                pos_word_2 = sent_pos[i + skip + 1]
                cond = (pos_word_1 in ('NOUN', 'ADJ'), pos_word_2 == 'NOUN')
                if not all(cond):
                    handle_failed_pairing(i, skip, num_paired, sent, sent_paired, to_remove)
                    skip += num_paired - 1
                    continue
            
            # trigrams: noun/adj, all types, noun/adj
            elif num_paired == 3:
                pos_word_1 = sent_pos[i + skip]
                pos_word_2 = sent_pos[i + skip + 1]
                pos_word_3 = sent_pos[i + skip + 2]
                cond = (pos_word_1 in ('NOUN', 'ADJ'), pos_word_3 in ('NOUN', 'ADJ'))
                if not all(cond):
                    handle_failed_pairing(i, skip, num_paired, sent, sent_paired, to_remove)
                    skip += num_paired - 1
                    continue
        
            # num. of words to skip indexing over sent and sent_pos in the next iter
            skip += num_paired - 1
        
    # remove rejected pairs that are already split and added back individually
    if len(to_remove) > 0:
        for j in sorted(to_remove, reverse=True):
            del sent_paired[j]

**Test the filtering function:**

Test 1:

In [104]:
sent = ['liver', 'support', 'supports', 'liver', 'function', 'stimulate', 'des', 'intoxication', 'and', 'restore', 'liver', 'function', 'eliminate', 'harmful', 'metabolite']
print(sent)

['liver', 'support', 'supports', 'liver', 'function', 'stimulate', 'des', 'intoxication', 'and', 'restore', 'liver', 'function', 'eliminate', 'harmful', 'metabolite']


In [105]:
sent_pos = ['PROPN', 'PROPN', 'PROPN', 'NOUN', 'NOUN', 'VERB', 'X', 'NOUN', 'CCONJ', 'VERB', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN']
print(sent_pos)

['PROPN', 'PROPN', 'PROPN', 'NOUN', 'NOUN', 'VERB', 'X', 'NOUN', 'CCONJ', 'VERB', 'NOUN', 'NOUN', 'VERB', 'ADJ', 'NOUN']


In [106]:
sent_paired = ['liver', 'support', 'supports', 'liver_function', 'stimulate_des_intoxication_and_restore', 'liver_function', 'eliminate', 'harmful', 'metabolite']
print(sent_paired)

['liver', 'support', 'supports', 'liver_function', 'stimulate_des_intoxication_and_restore', 'liver_function', 'eliminate', 'harmful', 'metabolite']


In [107]:
filter_pairs(sent, sent_paired, sent_pos)

In [108]:
# Expected output:
print(['liver', 'support', 'supports', 'liver_function', 'liver_function', 'eliminate', 'harmful', 'metabolite', 'stimulate', 'des', 'intoxication', 'and', 'restore'])

['liver', 'support', 'supports', 'liver_function', 'liver_function', 'eliminate', 'harmful', 'metabolite', 'stimulate', 'des', 'intoxication', 'and', 'restore']


In [109]:
print(sent_paired)

['liver', 'support', 'supports', 'liver_function', 'liver_function', 'eliminate', 'harmful', 'metabolite', 'stimulate', 'des', 'intoxication', 'and', 'restore']


Test 2:

In [110]:
sent = ['-PRON-', 'have', 'a', 'lot', 'more', 'energy', 'and', 'have', 'not', 'be', 'sick', 'at', 'all']
print(sent)

['-PRON-', 'have', 'a', 'lot', 'more', 'energy', 'and', 'have', 'not', 'be', 'sick', 'at', 'all']


In [111]:
sent_pos = ['PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'ADV', 'VERB', 'ADJ', 'ADV', 'ADV']
print(sent_pos)

['PRON', 'VERB', 'DET', 'NOUN', 'ADJ', 'NOUN', 'CCONJ', 'VERB', 'ADV', 'VERB', 'ADJ', 'ADV', 'ADV']


In [112]:
sent_paired = ['-PRON-', 'have', 'a_lot', 'more_energy', 'and', 'have', 'not', 'be', 'sick', 'at_all']
print(sent_paired)

['-PRON-', 'have', 'a_lot', 'more_energy', 'and', 'have', 'not', 'be', 'sick', 'at_all']


In [113]:
filter_pairs(sent, sent_paired, sent_pos)

In [114]:
print(sent_paired)

['-PRON-', 'have', 'more_energy', 'and', 'have', 'not', 'be', 'sick', 'a', 'lot', 'at', 'all']


### Filter Phrases

In [115]:
unigram_sents_pos_df.head()

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review,has_paired_words
0,1,1,[bioavailability...,"[PROPN, VERB, DE...",[bioavailability...,0
1,1,2,"[-PRON-, start, ...","[PRON, VERB, VER...","[-PRON-, start, ...",1
2,1,3,"[do, some, resea...","[VERB, DET, NOUN...","[do, some, resea...",1
3,1,4,"[the, ez, melt, ...","[DET, ADP, NOUN,...","[the, ez, melt, ...",0
4,1,5,"[this, product, ...","[DET, NOUN, VERB...","[this_product, h...",1


In [116]:
preprocessed_reviews = unigram_sents_pos_df.preprocessed_review.tolist()
unigram_sentences = unigram_sents_pos_df.unigram_sentences.tolist()
unigram_pos = unigram_sents_pos_df.unigram_pos.tolist()
has_paired_words = unigram_sents_pos_df.has_paired_words.tolist()

In [117]:
# get rid of paired words from the corpus which
# (1) have more than 3 words joined
# (2) bigrams not in the format: noun/adj, noun
# (3) trigrams not in the format: noun/adj, all types, noun/adj
for i in tqdm(range(len(preprocessed_reviews))):
    if has_paired_words[i] == 1:
        filter_pairs(sent=unigram_sentences[i], sent_paired=preprocessed_reviews[i], sent_pos=unigram_pos[i])

100%|██████████| 1241826/1241826 [00:03<00:00, 393305.14it/s]


In [118]:
# save picked dataframe to S3.  Pickle format allows the columns to store lists
save_df_s3(unigram_sents_pos_df, bucket_name, filepath='amazon_reviews/kk/preprocessed_reviews_v2.pkl', filetype='pickle')

In [5]:
# load from the pickled dataframe on S3
unigram_sents_pos_df = load_df_s3(bucket_name, filepath='amazon_reviews/kk/preprocessed_reviews_v2.pkl', filetype='pickle')

In [119]:
unigram_sents_pos_df.head()

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review,has_paired_words
0,1,1,[bioavailability...,"[PROPN, VERB, DE...",[bioavailability...,0
1,1,2,"[-PRON-, start, ...","[PRON, VERB, VER...","[-PRON-, start, ...",1
2,1,3,"[do, some, resea...","[VERB, DET, NOUN...","[do, some, resea...",1
3,1,4,"[the, ez, melt, ...","[DET, ADP, NOUN,...","[the, ez, melt, ...",0
4,1,5,"[this, product, ...","[DET, NOUN, VERB...","[have, the, grea...",1


In [120]:
unigram_sents_pos_df.shape

(1241826, 6)

In [121]:
preprocessed_review_updated = unigram_sents_pos_df.preprocessed_review.tolist()

In [122]:
len(preprocessed_review_updated)

1241826

In [123]:
preprocessed_review_updated[:3]

[['bioavailability', 'be', 'the', 'key'],
 ['-PRON-',
  'start',
  'take',
  'this',
  'after',
  'both',
  '-PRON-',
  'parent',
  'die',
  'of',
  'cancer',
  'as',
  '-PRON-',
  'suppose',
  'to',
  'enhance',
  '-PRON-',
  'immune_system',
  'the',
  'story',
  'on',
  'on',
  'resveratrol',
  'be',
  '60',
  'minutes',
  'incredibly',
  'inspiring'],
 ['do',
  'some',
  'research',
  'on',
  'the',
  'internet',
  '-PRON-',
  'be',
  'indicate',
  'that',
  'take',
  'resveratrol',
  'in',
  'lozenge',
  'form',
  'be',
  'preferable',
  'as',
  '-PRON-',
  'be',
  'by',
  'stomach',
  'acid',
  'break',
  'down']]

In [124]:
gramlist_updated = [word for sent in preprocessed_review_updated for word in sent if '_' in word]

In [125]:
paired_words_frq_updated = Counter(gramlist_updated)
paired_words_frq_updated.most_common(100)

[('fish_oil', 10809),
 ('side_effect', 4969),
 ('high_quality', 3908),
 ('vitamin_d', 3852),
 ('immune_system', 3374),
 ('long_time', 3112),
 ('vitamin_c', 2660),
 ('blood_pressure', 2457),
 ('energy_level', 2408),
 ('anyone_who', 2207),
 ('joint_pain', 1932),
 ('weight_loss', 1891),
 ('hot_flash', 1758),
 ('health_benefit', 1577),
 ('blood_sugar', 1465),
 ('digestive_system', 1354),
 ('krill_oil', 1312),
 ('people_who', 1227),
 ('vitamin_e', 1203),
 ('customer_service', 1112),
 ('long_term', 1076),
 ('acid_reflux', 1044),
 ('health_food_store', 1038),
 ('little_bit', 1038),
 ('fast_shipping', 1035),
 ('blood_test', 956),
 ('big_difference', 951),
 ('huge_difference', 950),
 ('reasonable_price', 950),
 ('coconut_oil', 928),
 ('second_bottle', 908),
 ('small_amount', 867),
 ('multi_vitamin', 808),
 ('green_tea', 807),
 ('daily_basis', 788),
 ('sore_throat', 785),
 ('local_health', 784),
 ('digestive_issue', 781),
 ('food_store', 763),
 ('soft_gel', 753),
 ('love_it!.', 725),
 ('leg_cram

In [127]:
len(paired_words_frq_updated)   # final number of cleaned-up paired words in the specified phrase format

27137

#### Final Clean-up: Remove Stop Words

In [128]:
unigram_sents_pos_df.head()

,review_number,sentence_number,unigram_sentences,unigram_pos,preprocessed_review,has_paired_words
0,1,1,[bioavailability...,"[PROPN, VERB, DE...",[bioavailability...,0
1,1,2,"[-PRON-, start, ...","[PRON, VERB, VER...","[-PRON-, start, ...",1
2,1,3,"[do, some, resea...","[VERB, DET, NOUN...","[do, some, resea...",1
3,1,4,"[the, ez, melt, ...","[DET, ADP, NOUN,...","[the, ez, melt, ...",0
4,1,5,"[this, product, ...","[DET, NOUN, VERB...","[have, the, grea...",1


In [129]:
unigram_sents_pos_df.shape

(1241826, 6)

In [130]:
preprocessed_review_final = [[word for word in sentence if word not in nlp.Defaults.stop_words] for sentence in preprocessed_review_updated]

In [131]:
unigram_sents_pos_df.drop(['preprocessed_review'], axis=1, inplace=True)
unigram_sents_pos_df['preprocessed_review'] = preprocessed_review_final
unigram_sents_pos_df.head()

,review_number,sentence_number,unigram_sentences,unigram_pos,has_paired_words,preprocessed_review
0,1,1,[bioavailability...,"[PROPN, VERB, DE...",0,[bioavailability...
1,1,2,"[-PRON-, start, ...","[PRON, VERB, VER...",1,"[-PRON-, start, ..."
2,1,3,"[do, some, resea...","[VERB, DET, NOUN...",1,"[research, inter..."
3,1,4,"[the, ez, melt, ...","[DET, ADP, NOUN,...",0,"[ez, melt, formu..."
4,1,5,"[this, product, ...","[DET, NOUN, VERB...",1,"[great, side_eff..."


In [132]:
# save picked dataframe to S3.  Pickle format allows the columns to store lists
save_df_s3(unigram_sents_pos_df, bucket_name, filepath='amazon_reviews/kk/preprocessed_reviews_v3.pkl', filetype='pickle')

In [5]:
# load from the pickled dataframe on S3
unigram_sents_pos_df = load_df_s3(bucket_name, filepath='amazon_reviews/kk/preprocessed_reviews_v3.pkl', filetype='pickle')

# Training the LDA Model

In [11]:
unigram_sents_pos_df.head(10)

,review_number,sentence_number,unigram_sentences,unigram_pos,has_paired_words,preprocessed_review
0,1,1,"[bioavailability, be, the, key]","[PROPN, VERB, DET, NOUN]",0,"[bioavailability, key]"
1,1,2,"[-PRON-, start, take, this, after, both, -PRON...","[PRON, VERB, VERB, DET, ADP, CCONJ, ADJ, NOUN,...",1,"[-PRON-, start, -PRON-, parent, die, cancer, -..."
2,1,3,"[do, some, research, on, the, internet, -PRON-...","[VERB, DET, NOUN, ADP, DET, NOUN, PRON, VERB, ...",1,"[research, internet, -PRON-, indicate, resvera..."
3,1,4,"[the, ez, melt, formula, recommend, in, anothe...","[DET, ADP, NOUN, NOUN, VERB, ADP, DET, NOUN, V...",0,"[ez, melt, formula, recommend, review, ok, -PR..."
4,1,5,"[this, product, have, the, great, side, effect...","[DET, NOUN, VERB, DET, ADJ, NOUN, NOUN, ADP, V...",1,"[great, side_effect, -PRON-, -PRON-, cold, sor..."
5,1,6,"[soon, after, start, to, take, -PRON-, every, ...","[ADV, ADP, VERB, PART, VERB, PRON, DET, NOUN, ...",1,"[soon, start, -PRON-, -PRON-, start, come, col..."
6,1,7,"[but, -PRON-, never, do, get, as, sick, as, an...","[CCONJ, PRON, ADV, VERB, VERB, ADV, ADJ, ADP, ...",1,"[-PRON-, sick, anticipate, taking, reason, -PR..."
7,1,8,"[since, then, -PRON-, have, have, no, cold, or...","[ADP, ADV, PRON, VERB, VERB, DET, NOUN, CCONJ,...",1,"[-PRON-, cold, sore_throat, -PRON-, great]"
8,1,9,"[-PRON-, recommend, this, product, to, everyon...","[PRON, VERB, DET, NOUN, ADP, NOUN, PRON, VERB,...",1,"[-PRON-, recommend, -PRON-, know, -PRON-, gift..."
9,2,10,"[-PRON-, can, not, find, this, product, any, l...","[PRON, VERB, ADV, VERB, DET, NOUN, ADV, ADV, C...",1,"[-PRON-, find, longer, -PRON-, wish, -PRON-, p..."


In [12]:
unigram_sents_pos_df.drop(['unigram_sentences', 'unigram_pos', 'has_paired_words'], axis=1, inplace=True)

In [13]:
# Remove '-PRON-' from the token list
unigram_sents_pos_df.preprocessed_review = unigram_sents_pos_df.preprocessed_review.map(lambda x: [word for word in x if word != '-PRON-'])

In [14]:
# get rid of tokens that do not contain any alphabets
def has_any_alpha(word):
    return any([char for char in word if char.isalpha()])

unigram_sents_pos_df.preprocessed_review = unigram_sents_pos_df.preprocessed_review.map(lambda x: [word for word in x if has_any_alpha(word)])

In [15]:
# remove one-character tokens
unigram_sents_pos_df.preprocessed_review = unigram_sents_pos_df.preprocessed_review.map(lambda x: [word for word in x if len(word) > 1])

In [16]:
unigram_sents_pos_df.head()

,review_number,sentence_number,preprocessed_review
0,1,1,"[bioavailability, key]"
1,1,2,"[start, parent, die, cancer, suppose, enhance,..."
2,1,3,"[research, internet, indicate, resveratrol, lo..."
3,1,4,"[ez, melt, formula, recommend, review, ok, dis..."
4,1,5,"[great, side_effect, cold, sore_throat, product]"


In [41]:
review_num = unigram_sents_pos_df.review_number.tolist()

In [18]:
tokenized_reviews = unigram_sents_pos_df.preprocessed_review.tolist()

In [19]:
print(tokenized_reviews[:3])

[['bioavailability', 'key'], ['start', 'parent', 'die', 'cancer', 'suppose', 'enhance', 'immune_system', 'story', 'resveratrol', 'minutes', 'incredibly', 'inspiring'], ['research', 'internet', 'indicate', 'resveratrol', 'lozenge', 'form', 'preferable', 'stomach', 'acid', 'break']]


In [20]:
tokenized_reviews = unigram_sents_pos_df.groupby(('review_number'))['preprocessed_review'].sum()

In [21]:
tokenized_reviews = tokenized_reviews.tolist()

In [22]:
len(tokenized_reviews)

217530

In [23]:
print(tokenized_reviews[0])

['bioavailability', 'key', 'start', 'parent', 'die', 'cancer', 'suppose', 'enhance', 'immune_system', 'story', 'resveratrol', 'minutes', 'incredibly', 'inspiring', 'research', 'internet', 'indicate', 'resveratrol', 'lozenge', 'form', 'preferable', 'stomach', 'acid', 'break', 'ez', 'melt', 'formula', 'recommend', 'review', 'ok', 'dissolve', 'mouth', 'quickly', 'lozenge', 'formula', 'dissolve', 'slowly', 'preferable', 'accord', 'research', 'great', 'side_effect', 'cold', 'sore_throat', 'product', 'soon', 'start', 'start', 'come', 'cold', 'usual', 'symptom', 'anticipate', 'sick', 'day', 'usual', 'pattern', 'day', 'sick', 'anticipate', 'taking', 'reason', 'come', 'product', 'cold', 'sore_throat', 'great', 'recommend', 'know', 'gift', 'family', 'product']


In [7]:
# save_df_s3(tokenized_reviews, bucket_name, filepath='amazon_reviews/kk/tokenized_reviews_v1.pkl', filetype='pickle')
tokenized_reviews = load_df_s3(bucket_name, filepath='amazon_reviews/kk/tokenized_reviews_v1.pkl', filetype='pickle')

In [16]:
%%time
# we need to learn the full vocabulary of the corpus to be modeled
# learn the dictionary by iterating over all of the reviews
vocab_dictionary = Dictionary(tokenized_reviews)

CPU times: user 9.18 s, sys: 12 ms, total: 9.19 s
Wall time: 9.19 s


In [17]:
print(vocab_dictionary)

Dictionary(133384 unique tokens: ['accord', 'acid', 'anticipate', 'bioavailability', 'break']...)


In [18]:
once_ids = [tokenid for tokenid, docfreq in iteritems(vocab_dictionary.dfs) if docfreq == 1]
vocab_dictionary.filter_tokens(bad_ids=once_ids)  # remove words that appear only once

In [19]:
print(vocab_dictionary)

Dictionary(42235 unique tokens: ['accord', 'acid', 'anticipate', 'bioavailability', 'break']...)


In [22]:
# filter tokens that are very rare or too common from
# the dictionary (filter_extremes) and reassign integer ids (compactify)
# here we remove tokens which do not appear in at least 10 reviews
vocab_dictionary.filter_extremes(no_below=10, no_above=0.5)
vocab_dictionary.compactify()   # remove gaps in id sequence after words that were removed

In [8]:
print(vocab_dictionary)

Dictionary(13850 unique tokens: ['accord', 'acid', 'anticipate', 'bioavailability', 'break']...)


In [26]:
save_df_s3(vocab_dictionary, bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_v3.dict', filetype='pickle')

In [7]:
vocab_dictionary = load_df_s3(bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_v3.dict', filetype='pickle')
# vocab_dictionary = Dictionary.load('../vocab_dictionary.dict')  # load the finished dictionary from disk

In [6]:
tokenized_reviews = load_df_s3(bucket_name, filepath='amazon_reviews/kk/tokenized_reviews_v1.pkl', filetype='pickle')

In [9]:
# bag-of-words representation of the corpus/ doc-term matrix
bow_corpus = [vocab_dictionary.doc2bow(review) for review in tokenized_reviews]

## First Order LDA Model

In [27]:
%%time

# Set training parameters.
num_topics_list = [4]
chunksize = 7000    # number of docs processed at a time
passes = 3
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = None           # symmetric prior
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda = LdaModel(bow_corpus, num_topics=num_topics, id2word=vocab_dictionary, chunksize=chunksize, 
                   passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda, bow_corpus, vocab_dictionary)
    plot_fname = 'pyLDAvis_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/1 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
INFO : using symmetric eta at 0.25
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 4 topics, 3 passes over the supplied corpus of 217530 documents, updating model once every 7000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #7000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6713/7000 documents converged within 400 iterations
INFO : optimized alpha [0.072370008, 0.082538292, 0.11915907, 0.15756291]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.072): 0.023*"pedometer" + 0.019*"step" + 0.019*"day" + 0.016*"use" + 0.015*"great" + 0.015*"good" + 0.015*"work" + 0.015*"like" + 0.010*"product" + 0.009*"walk"
INFO : topic #1 (0.083): 0.028*"pedomet

DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.12343414, 0.12966594, 0.11762695, 0.079292007]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.123): 0.018*"help" + 0.016*"work" + 0.016*"day" + 0.014*"product" + 0.014*"good" + 0.011*"feel" + 0.011*"try" + 0.010*"use" + 0.010*"year" + 0.009*"start"
INFO : topic #1 (0.130): 0.030*"good" + 0.030*"product" + 0.025*"great" + 0.020*"taste" + 0.016*"use" + 0.013*"like" + 0.010*"price" + 0.009*"work" + 0.009*"find" + 0.009*"supplement"
INFO : topic #2 (0.118): 0.051*"product" + 0.029*"use" + 0.027*"good" + 0.025*"great" + 0.016*"work" + 0.016*"skin" + 0.013*"love" + 0.011*"recommend" + 0.010*"year" + 0.010*"oil"
INFO : topic #3 (0.079): 0.026*"product" + 0.023*"use" + 0.020*"day" + 0.020*"work" + 0.013*"good" + 0.012*"pain" + 0.012*"great" + 0.011*"time" + 0.011*"uti" + 0.010*"yea

INFO : topic #2 (0.133): 0.058*"product" + 0.031*"use" + 0.031*"great" + 0.027*"good" + 0.016*"work" + 0.014*"skin" + 0.013*"love" + 0.012*"recommend" + 0.010*"hair" + 0.010*"buy"
INFO : topic #3 (0.085): 0.022*"product" + 0.020*"day" + 0.020*"use" + 0.020*"work" + 0.019*"pain" + 0.010*"good" + 0.010*"time" + 0.010*"great" + 0.009*"help" + 0.009*"year"
INFO : topic diff=0.253378, rho=0.258199
INFO : PROGRESS: pass 0, at document #112000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.16724987, 0.16620769, 0.13625547, 0.085355461]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.167): 0.018*"help" + 0.016*"work" + 0.016*"day" + 0.014*"product" + 0.012*"good" + 0.011*"feel" + 0.010*"try" + 0.010*"supplement" + 0.010*"year" + 0.010*"use"
INFO : topic #1 (0.166): 0.034*"good" + 0.026*"product" + 0.024*"great" + 0.020*"t

DEBUG : 7000/7000 documents converged within 400 iterations
INFO : optimized alpha [0.17716767, 0.18951592, 0.16166346, 0.090444118]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.177): 0.018*"help" + 0.016*"work" + 0.016*"day" + 0.015*"product" + 0.012*"good" + 0.012*"feel" + 0.010*"try" + 0.010*"year" + 0.010*"supplement" + 0.010*"use"
INFO : topic #1 (0.190): 0.034*"good" + 0.024*"taste" + 0.024*"great" + 0.023*"product" + 0.016*"like" + 0.013*"vitamin" + 0.013*"use" + 0.012*"easy" + 0.011*"love" + 0.010*"price"
INFO : topic #2 (0.162): 0.062*"product" + 0.031*"great" + 0.029*"use" + 0.028*"good" + 0.017*"hair" + 0.013*"love" + 0.013*"work" + 0.013*"skin" + 0.012*"recommend" + 0.012*"oil"
INFO : topic #3 (0.090): 0.023*"work" + 0.021*"day" + 0.020*"product" + 0.019*"use" + 0.017*"pain" + 0.012*"week" + 0.010*"start" + 0.010*"time" + 0.010*"help" + 0.009*"feel"
INFO : topic diff=0.197441, rho=0.208514
INFO : PROG

INFO : topic #2 (0.185): 0.069*"product" + 0.033*"great" + 0.028*"good" + 0.027*"use" + 0.016*"skin" + 0.013*"love" + 0.012*"recommend" + 0.012*"oil" + 0.012*"work" + 0.010*"price"
INFO : topic #3 (0.103): 0.023*"work" + 0.021*"product" + 0.020*"day" + 0.017*"use" + 0.015*"pain" + 0.014*"week" + 0.014*"feel" + 0.011*"try" + 0.011*"help" + 0.010*"start"
INFO : topic diff=0.186617, rho=0.182574
INFO : PROGRESS: pass 0, at document #217000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 7000/7000 documents converged within 400 iterations
INFO : optimized alpha [0.18876565, 0.19359982, 0.19071725, 0.10616958]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.189): 0.018*"help" + 0.016*"product" + 0.015*"work" + 0.014*"day" + 0.014*"feel" + 0.014*"supplement" + 0.013*"good" + 0.011*"try" + 0.009*"recommend" + 0.009*"use"
INFO : topic #1 (0.194): 0.030*"good" + 0.024*"great" + 0.023*"taste" + 0.022*

...

INFO : CorpusAccumulator accumulated stats from 1000 documents
INFO : CorpusAccumulator accumulated stats from 2000 documents
INFO : CorpusAccumulator accumulated stats from 3000 documents
INFO : CorpusAccumulator accumulated stats from 4000 documents
INFO : CorpusAccumulator accumulated stats from 5000 documents
INFO : CorpusAccumulator accumulated stats from 6000 documents
INFO : CorpusAccumulator accumulated stats from 7000 documents
INFO : CorpusAccumulator accumulated stats from 8000 documents
INFO : CorpusAccumulator accumulated stats from 9000 documents
INFO : CorpusAccumulator accumulated stats from 10000 documents
INFO : CorpusAccumulator accumulated stats from 11000 documents
INFO : CorpusAccumulator accumulated stats from 12000 documents
INFO : CorpusAccumulator accumulated stats from 13000 documents
INFO : CorpusAccumulator accumulated stats from 14000 documents
INFO : CorpusAccumulator accumulated stats from 15000 documents
INFO : CorpusAccumulator accumulated stats from 1

INFO : CorpusAccumulator accumulated stats from 129000 documents
INFO : CorpusAccumulator accumulated stats from 130000 documents
INFO : CorpusAccumulator accumulated stats from 131000 documents
INFO : CorpusAccumulator accumulated stats from 132000 documents
INFO : CorpusAccumulator accumulated stats from 133000 documents
INFO : CorpusAccumulator accumulated stats from 134000 documents
INFO : CorpusAccumulator accumulated stats from 135000 documents
INFO : CorpusAccumulator accumulated stats from 136000 documents
INFO : CorpusAccumulator accumulated stats from 137000 documents
INFO : CorpusAccumulator accumulated stats from 138000 documents
INFO : CorpusAccumulator accumulated stats from 139000 documents
INFO : CorpusAccumulator accumulated stats from 140000 documents
INFO : CorpusAccumulator accumulated stats from 141000 documents
INFO : CorpusAccumulator accumulated stats from 142000 documents
INFO : CorpusAccumulator accumulated stats from 143000 documents
INFO : CorpusAccumulator 

Calculated coherence score...:  -2.16684973066


DEBUG : performing inference on a chunk of 217530 documents
DEBUG : 217520/217530 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 1/1 [11:12<00:00, 672.29s/it]

Saved html file:  pyLDAvis_4.html
CPU times: user 11min 12s, sys: 1.35 s, total: 11min 13s
Wall time: 11min 12s


In [15]:
%%time
# setting alpha and eta manually
# Set training parameters.
num_topics_list = [4]
chunksize = 7000    # number of docs processed at a time
passes = 5
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = 0.5e-2
alpha = 1e-2

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda = LdaMulticore(bow_corpus, num_topics=num_topics, id2word=vocab_dictionary, chunksize=chunksize, 
                   passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda, bow_corpus, vocab_dictionary)
    plot_fname = 'pyLDAvis_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/1 [00:00<?, ?it/s]/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/gensim/models/ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


Trained LDA for 4 topics...
Calculated coherence score...:  -1.94634373959


/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 1/1 [06:40<00:00, 400.83s/it]

Saved html file:  pyLDAvis_4.html
CPU times: user 6min 53s, sys: 25.9 s, total: 7min 19s
Wall time: 6min 40s


In [9]:
%%time

# Set training parameters.
num_topics_list = [6, 7]
chunksize = 5000    # number of docs processed at a time
passes = 2
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = None           # symmetric prior
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda = LdaModel(bow_corpus, num_topics=num_topics, id2word=vocab_dictionary, chunksize=chunksize, 
                   passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda, bow_corpus, vocab_dictionary)
    plot_fname = 'pyLDAvis_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/2 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667]
INFO : using symmetric eta at 0.16666666666666666
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 6 topics, 2 passes over the supplied corpus of 217530 documents, updating model once every 5000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #5000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 4885/5000 documents converged within 400 iterations
INFO : optimized alpha [0.081580229, 0.082519785, 0.10786959, 0.11670941, 0.10751451, 0.071371369]
DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #5 (0.071): 0.029*"work" + 0.020*"great" + 0.016*"use" + 0.015*"good" + 0.013*"product" + 0.012*"try" + 0.011*"buy" + 

INFO : topic diff=0.435455, rho=0.408248
INFO : PROGRESS: pass 0, at document #35000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 5000/5000 documents converged within 400 iterations
INFO : optimized alpha [0.10611774, 0.10967626, 0.12968247, 0.088895626, 0.060573161, 0.15795206]
DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #4 (0.061): 0.026*"use" + 0.018*"day" + 0.014*"easy" + 0.014*"love" + 0.013*"great" + 0.012*"work" + 0.011*"pedometer" + 0.011*"good" + 0.010*"product" + 0.008*"purchase"
INFO : topic #3 (0.089): 0.028*"product" + 0.024*"day" + 0.023*"use" + 0.016*"work" + 0.014*"good" + 0.013*"great" + 0.011*"time" + 0.011*"pain" + 0.009*"like" + 0.009*"year"
INFO : topic #1 (0.110): 0.037*"product" + 0.036*"good" + 0.033*"great" + 0.022*"taste" + 0.018*"price" + 0.015*"use" + 0.015*"like" + 0.012*"vitamin" + 0.011*"find" + 0.010*"recommend"
INFO : topic #2 (0.130): 0.049*"product" + 0.03

INFO : topic #4 (0.056): 0.022*"use" + 0.014*"day" + 0.010*"tablespoon" + 0.010*"love" + 0.010*"bladder" + 0.010*"work" + 0.009*"easy" + 0.007*"product" + 0.007*"good" + 0.007*"great"
INFO : topic #3 (0.103): 0.026*"product" + 0.023*"day" + 0.021*"pain" + 0.019*"use" + 0.018*"work" + 0.012*"good" + 0.010*"help" + 0.010*"year" + 0.010*"time" + 0.009*"great"
INFO : topic #2 (0.144): 0.057*"product" + 0.033*"use" + 0.026*"great" + 0.024*"good" + 0.018*"skin" + 0.014*"love" + 0.013*"work" + 0.012*"oil" + 0.011*"recommend" + 0.010*"year"
INFO : topic #1 (0.148): 0.045*"good" + 0.040*"product" + 0.035*"great" + 0.025*"taste" + 0.019*"price" + 0.016*"like" + 0.015*"use" + 0.011*"find" + 0.011*"brand" + 0.011*"love"
INFO : topic #5 (0.201): 0.025*"work" + 0.017*"product" + 0.017*"help" + 0.016*"feel" + 0.015*"try" + 0.015*"day" + 0.015*"use" + 0.014*"good" + 0.013*"start" + 0.012*"year"
INFO : topic diff=0.247012, rho=0.277350
INFO : PROGRESS: pass 0, at document #70000/217530
DEBUG : performi

INFO : topic #1 (0.191): 0.047*"good" + 0.037*"product" + 0.035*"great" + 0.024*"taste" + 0.020*"price" + 0.016*"use" + 0.016*"like" + 0.014*"vitamin" + 0.013*"brand" + 0.011*"find"
INFO : topic #5 (0.227): 0.027*"work" + 0.019*"feel" + 0.019*"day" + 0.017*"help" + 0.016*"product" + 0.015*"try" + 0.014*"use" + 0.014*"good" + 0.013*"start" + 0.012*"year"
INFO : topic diff=0.179252, rho=0.229416
INFO : PROGRESS: pass 0, at document #100000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 5000/5000 documents converged within 400 iterations
INFO : optimized alpha [0.17590527, 0.19929582, 0.1652644, 0.11814775, 0.056747705, 0.22837983]
DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #4 (0.057): 0.018*"use" + 0.017*"seed" + 0.017*"coal" + 0.010*"tablespoon" + 0.010*"eye" + 0.010*"day" + 0.007*"light" + 0.006*"easy" + 0.006*"love" + 0.006*"time"
INFO : topic #3 (0.118): 0.025*"pain" + 0.024*"product" + 0.

INFO : topic diff=0.139327, rho=0.200000
INFO : PROGRESS: pass 0, at document #130000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 5000/5000 documents converged within 400 iterations
INFO : optimized alpha [0.19771701, 0.26169908, 0.18924558, 0.12436403, 0.057545397, 0.24735048]
DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #4 (0.058): 0.013*"use" + 0.012*"seed" + 0.011*"eye" + 0.008*"day" + 0.008*"tablespoon" + 0.008*"osteoporosis" + 0.006*"orange" + 0.006*"second_bottle" + 0.006*"dry_eye" + 0.005*"easy"
INFO : topic #3 (0.124): 0.023*"pain" + 0.023*"product" + 0.021*"day" + 0.017*"work" + 0.017*"use" + 0.013*"help" + 0.011*"week" + 0.010*"year" + 0.009*"good" + 0.009*"start"
INFO : topic #0 (0.198): 0.021*"supplement" + 0.015*"help" + 0.013*"good" + 0.013*"product" + 0.011*"recommend" + 0.011*"doctor" + 0.009*"mg" + 0.009*"day" + 0.009*"'s" + 0.009*"level"
INFO : topic #5 (0.247): 0.029*"wo

...

INFO : CorpusAccumulator accumulated stats from 15000 documents
INFO : CorpusAccumulator accumulated stats from 16000 documents
INFO : CorpusAccumulator accumulated stats from 17000 documents
INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated sta

INFO : CorpusAccumulator accumulated stats from 143000 documents
INFO : CorpusAccumulator accumulated stats from 144000 documents
INFO : CorpusAccumulator accumulated stats from 145000 documents
INFO : CorpusAccumulator accumulated stats from 146000 documents
INFO : CorpusAccumulator accumulated stats from 147000 documents
INFO : CorpusAccumulator accumulated stats from 148000 documents
INFO : CorpusAccumulator accumulated stats from 149000 documents
INFO : CorpusAccumulator accumulated stats from 150000 documents
INFO : CorpusAccumulator accumulated stats from 151000 documents
INFO : CorpusAccumulator accumulated stats from 152000 documents
INFO : CorpusAccumulator accumulated stats from 153000 documents
INFO : CorpusAccumulator accumulated stats from 154000 documents
INFO : CorpusAccumulator accumulated stats from 155000 documents
INFO : CorpusAccumulator accumulated stats from 156000 documents
INFO : CorpusAccumulator accumulated stats from 157000 documents
INFO : CorpusAccumulator 

Calculated coherence score...:  -3.16515368355


DEBUG : performing inference on a chunk of 217530 documents
DEBUG : 217530/217530 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 2/2 [14:11<00:00, 425.73s/it]

Saved html file:  pyLDAvis_7.html
CPU times: user 14min 10s, sys: 3.03 s, total: 14min 13s
Wall time: 14min 11s


In [10]:
%%time

# Set training parameters.
num_topics_list = [8]
chunksize = 5000    # number of docs processed at a time
passes = 2
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = None           # symmetric prior
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda = LdaModel(bow_corpus, num_topics=num_topics, id2word=vocab_dictionary, chunksize=chunksize, 
                   passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda, bow_corpus, vocab_dictionary)
    plot_fname = 'pyLDAvis_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/1 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125, 0.125]
INFO : using symmetric eta at 0.125
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 8 topics, 2 passes over the supplied corpus of 217530 documents, updating model once every 5000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #5000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 4910/5000 documents converged within 400 iterations
INFO : optimized alpha [0.076057054, 0.076216474, 0.093472309, 0.097475961, 0.090470284, 0.071344614, 0.088717267, 0.074614719]
DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #5 (0.071): 0.026*"work" + 0.020*"use" + 0.018*"good" + 0.016*"great" + 0.013*"product" + 0.012*"buy" + 0.011*"try" + 0

INFO : topic #5 (0.148): 0.020*"work" + 0.018*"good" + 0.018*"product" + 0.017*"use" + 0.015*"help" + 0.013*"feel" + 0.013*"try" + 0.012*"year" + 0.012*"great" + 0.010*"start"
INFO : topic diff=0.425750, rho=0.408248
INFO : PROGRESS: pass 0, at document #35000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 4999/5000 documents converged within 400 iterations
INFO : optimized alpha [0.089173198, 0.10862322, 0.11195778, 0.086337395, 0.05848914, 0.15853426, 0.083004758, 0.05961604]
DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #4 (0.058): 0.031*"use" + 0.019*"day" + 0.017*"love" + 0.013*"product" + 0.012*"work" + 0.011*"good" + 0.011*"great" + 0.010*"easy" + 0.010*"pedometer" + 0.009*"tablespoon"
INFO : topic #7 (0.060): 0.014*"find" + 0.014*"swanson" + 0.012*"good" + 0.011*"day" + 0.010*"buy" + 0.010*"product" + 0.010*"great" + 0.010*"try" + 0.010*"purchase" + 0.009*"amazon"
INFO : topic #1 (0.109

INFO : topic diff=0.238265, rho=0.288675
INFO : PROGRESS: pass 0, at document #65000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 5000/5000 documents converged within 400 iterations
INFO : optimized alpha [0.10970945, 0.14724971, 0.11536144, 0.10666095, 0.054656278, 0.1943728, 0.097723678, 0.062685505]
DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #4 (0.055): 0.028*"use" + 0.020*"hair" + 0.014*"love" + 0.013*"day" + 0.011*"seed" + 0.011*"tablespoon" + 0.010*"bladder" + 0.010*"product" + 0.010*"eye" + 0.009*"work"
INFO : topic #7 (0.063): 0.018*"find" + 0.016*"amazon" + 0.015*"calm" + 0.011*"good" + 0.010*"buy" + 0.010*"anxiety" + 0.010*"try" + 0.010*"brand" + 0.009*"product" + 0.009*"'s"
INFO : topic #2 (0.115): 0.046*"product" + 0.037*"use" + 0.025*"skin" + 0.023*"great" + 0.022*"good" + 0.016*"love" + 0.015*"oil" + 0.013*"work" + 0.009*"year" + 0.009*"recommend"
INFO : topic #1 (0.147): 0.0

DEBUG : updating topics
INFO : merging changes from 5000 documents into a model of 217530 documents
INFO : topic #4 (0.055): 0.038*"hair" + 0.028*"eye" + 0.022*"use" + 0.018*"lutein" + 0.016*"grow" + 0.012*"seed" + 0.010*"tablespoon" + 0.009*"love" + 0.009*"day" + 0.007*"product"
INFO : topic #7 (0.068): 0.020*"find" + 0.018*"amazon" + 0.012*"brand" + 0.011*"buy" + 0.011*"coq10" + 0.010*"good" + 0.009*"save" + 0.009*"'s" + 0.008*"calm" + 0.008*"anxiety"
INFO : topic #0 (0.130): 0.018*"help" + 0.015*"supplement" + 0.012*"day" + 0.010*"good" + 0.010*"product" + 0.009*"body" + 0.009*"doctor" + 0.009*"health" + 0.009*"recommend" + 0.008*"work"
INFO : topic #1 (0.193): 0.068*"product" + 0.057*"good" + 0.046*"great" + 0.025*"price" + 0.020*"use" + 0.016*"taste" + 0.014*"recommend" + 0.013*"brand" + 0.012*"buy" + 0.012*"find"
INFO : topic #5 (0.216): 0.025*"work" + 0.019*"feel" + 0.017*"help" + 0.017*"good" + 0.017*"product" + 0.016*"use" + 0.015*"try" + 0.015*"day" + 0.012*"year" + 0.011*"st

INFO : topic #7 (0.074): 0.023*"find" + 0.021*"amazon" + 0.014*"buy" + 0.013*"brand" + 0.011*"save" + 0.011*"'s" + 0.010*"anxiety" + 0.009*"order" + 0.009*"purchase" + 0.008*"good"
INFO : topic #6 (0.144): 0.025*"taste" + 0.021*"like" + 0.019*"pill" + 0.019*"easy" + 0.017*"vitamin" + 0.017*"good" + 0.016*"fish_oil" + 0.014*"great" + 0.014*"swallow" + 0.013*"supplement"
INFO : topic #5 (0.223): 0.027*"work" + 0.020*"feel" + 0.018*"help" + 0.017*"good" + 0.017*"day" + 0.016*"try" + 0.016*"use" + 0.016*"product" + 0.012*"year" + 0.011*"start"
INFO : topic #1 (0.244): 0.074*"product" + 0.061*"good" + 0.049*"great" + 0.028*"price" + 0.020*"use" + 0.016*"taste" + 0.016*"recommend" + 0.015*"buy" + 0.014*"brand" + 0.012*"find"
INFO : topic diff=0.127501, rho=0.200000
INFO : PROGRESS: pass 0, at document #130000/217530
DEBUG : performing inference on a chunk of 5000 documents
DEBUG : 5000/5000 documents converged within 400 iterations
INFO : optimized alpha [0.14270869, 0.25511378, 0.1239526, 0

...

INFO : CorpusAccumulator accumulated stats from 14000 documents
INFO : CorpusAccumulator accumulated stats from 15000 documents
INFO : CorpusAccumulator accumulated stats from 16000 documents
INFO : CorpusAccumulator accumulated stats from 17000 documents
INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated sta

INFO : CorpusAccumulator accumulated stats from 142000 documents
INFO : CorpusAccumulator accumulated stats from 143000 documents
INFO : CorpusAccumulator accumulated stats from 144000 documents
INFO : CorpusAccumulator accumulated stats from 145000 documents
INFO : CorpusAccumulator accumulated stats from 146000 documents
INFO : CorpusAccumulator accumulated stats from 147000 documents
INFO : CorpusAccumulator accumulated stats from 148000 documents
INFO : CorpusAccumulator accumulated stats from 149000 documents
INFO : CorpusAccumulator accumulated stats from 150000 documents
INFO : CorpusAccumulator accumulated stats from 151000 documents
INFO : CorpusAccumulator accumulated stats from 152000 documents
INFO : CorpusAccumulator accumulated stats from 153000 documents
INFO : CorpusAccumulator accumulated stats from 154000 documents
INFO : CorpusAccumulator accumulated stats from 155000 documents
INFO : CorpusAccumulator accumulated stats from 156000 documents
INFO : CorpusAccumulator 

Calculated coherence score...:  -2.69024937681


DEBUG : performing inference on a chunk of 217530 documents
DEBUG : 217530/217530 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 1/1 [07:26<00:00, 446.92s/it]

Saved html file:  pyLDAvis_8.html
CPU times: user 7min 26s, sys: 1.44 s, total: 7min 27s
Wall time: 7min 26s


## Second Order Topic Model

We start with exploring the 4-topic model because it showed clear separation of topics.

In [11]:
%%time

# Set training parameters.
num_topics_list = [4]
chunksize = 7000    # number of docs processed at a time
passes = 2
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = None           # symmetric prior
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda = LdaModel(bow_corpus, num_topics=num_topics, id2word=vocab_dictionary, chunksize=chunksize, 
                   passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    print('Trained LDA for {} topics...'.format(num_topics))
    save_df_s3(lda, bucket_name, filepath='amazon_reviews/kk/lda_tier2_topics_4.pkl', filetype='pickle')
    
    # performance metric
    cm = CoherenceModel(model=lda, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda, bow_corpus, vocab_dictionary)
    plot_fname = 'pyLDAvis_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/1 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.25, 0.25, 0.25, 0.25]
INFO : using symmetric eta at 0.25
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 4 topics, 2 passes over the supplied corpus of 217530 documents, updating model once every 7000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #7000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6713/7000 documents converged within 400 iterations
INFO : optimized alpha [0.072370008, 0.082538292, 0.11915907, 0.15756291]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.072): 0.023*"pedometer" + 0.019*"step" + 0.019*"day" + 0.016*"use" + 0.015*"great" + 0.015*"good" + 0.015*"work" + 0.015*"like" + 0.010*"product" + 0.009*"walk"
INFO : topic #1 (0.083): 0.028*"pedomet

DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.12343414, 0.12966594, 0.11762695, 0.079292007]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.123): 0.018*"help" + 0.016*"work" + 0.016*"day" + 0.014*"product" + 0.014*"good" + 0.011*"feel" + 0.011*"try" + 0.010*"use" + 0.010*"year" + 0.009*"start"
INFO : topic #1 (0.130): 0.030*"good" + 0.030*"product" + 0.025*"great" + 0.020*"taste" + 0.016*"use" + 0.013*"like" + 0.010*"price" + 0.009*"work" + 0.009*"find" + 0.009*"supplement"
INFO : topic #2 (0.118): 0.051*"product" + 0.029*"use" + 0.027*"good" + 0.025*"great" + 0.016*"work" + 0.016*"skin" + 0.013*"love" + 0.011*"recommend" + 0.010*"year" + 0.010*"oil"
INFO : topic #3 (0.079): 0.026*"product" + 0.023*"use" + 0.020*"day" + 0.020*"work" + 0.013*"good" + 0.012*"pain" + 0.012*"great" + 0.011*"time" + 0.011*"uti" + 0.010*"yea

INFO : topic #2 (0.133): 0.058*"product" + 0.031*"use" + 0.031*"great" + 0.027*"good" + 0.016*"work" + 0.014*"skin" + 0.013*"love" + 0.012*"recommend" + 0.010*"hair" + 0.010*"buy"
INFO : topic #3 (0.085): 0.022*"product" + 0.020*"day" + 0.020*"use" + 0.020*"work" + 0.019*"pain" + 0.010*"good" + 0.010*"time" + 0.010*"great" + 0.009*"help" + 0.009*"year"
INFO : topic diff=0.253378, rho=0.258199
INFO : PROGRESS: pass 0, at document #112000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.16724987, 0.16620769, 0.13625547, 0.085355461]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.167): 0.018*"help" + 0.016*"work" + 0.016*"day" + 0.014*"product" + 0.012*"good" + 0.011*"feel" + 0.010*"try" + 0.010*"supplement" + 0.010*"year" + 0.010*"use"
INFO : topic #1 (0.166): 0.034*"good" + 0.026*"product" + 0.024*"great" + 0.020*"t

DEBUG : 7000/7000 documents converged within 400 iterations
INFO : optimized alpha [0.17716767, 0.18951592, 0.16166346, 0.090444118]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.177): 0.018*"help" + 0.016*"work" + 0.016*"day" + 0.015*"product" + 0.012*"good" + 0.012*"feel" + 0.010*"try" + 0.010*"year" + 0.010*"supplement" + 0.010*"use"
INFO : topic #1 (0.190): 0.034*"good" + 0.024*"taste" + 0.024*"great" + 0.023*"product" + 0.016*"like" + 0.013*"vitamin" + 0.013*"use" + 0.012*"easy" + 0.011*"love" + 0.010*"price"
INFO : topic #2 (0.162): 0.062*"product" + 0.031*"great" + 0.029*"use" + 0.028*"good" + 0.017*"hair" + 0.013*"love" + 0.013*"work" + 0.013*"skin" + 0.012*"recommend" + 0.012*"oil"
INFO : topic #3 (0.090): 0.023*"work" + 0.021*"day" + 0.020*"product" + 0.019*"use" + 0.017*"pain" + 0.012*"week" + 0.010*"start" + 0.010*"time" + 0.010*"help" + 0.009*"feel"
INFO : topic diff=0.197441, rho=0.208514
INFO : PROG

INFO : topic #2 (0.185): 0.069*"product" + 0.033*"great" + 0.028*"good" + 0.027*"use" + 0.016*"skin" + 0.013*"love" + 0.012*"recommend" + 0.012*"oil" + 0.012*"work" + 0.010*"price"
INFO : topic #3 (0.103): 0.023*"work" + 0.021*"product" + 0.020*"day" + 0.017*"use" + 0.015*"pain" + 0.014*"week" + 0.014*"feel" + 0.011*"try" + 0.011*"help" + 0.010*"start"
INFO : topic diff=0.186617, rho=0.182574
INFO : PROGRESS: pass 0, at document #217000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 7000/7000 documents converged within 400 iterations
INFO : optimized alpha [0.18876565, 0.19359982, 0.19071725, 0.10616958]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.189): 0.018*"help" + 0.016*"product" + 0.015*"work" + 0.014*"day" + 0.014*"feel" + 0.014*"supplement" + 0.013*"good" + 0.011*"try" + 0.009*"recommend" + 0.009*"use"
INFO : topic #1 (0.194): 0.030*"good" + 0.024*"great" + 0.023*"taste" + 0.022*

...

INFO : topic #2 (0.188): 0.070*"product" + 0.036*"great" + 0.031*"use" + 0.031*"good" + 0.015*"skin" + 0.015*"price" + 0.014*"work" + 0.013*"love" + 0.013*"recommend" + 0.012*"buy"
INFO : topic #3 (0.118): 0.023*"day" + 0.023*"work" + 0.019*"pain" + 0.017*"use" + 0.016*"product" + 0.012*"week" + 0.011*"feel" + 0.011*"help" + 0.011*"start" + 0.010*"try"
INFO : topic diff=0.142588, rho=0.173878
INFO : PROGRESS: pass 1, at document #98000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 7000/7000 documents converged within 400 iterations
INFO : optimized alpha [0.23718999, 0.19094783, 0.18879527, 0.11786384]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.237): 0.019*"help" + 0.017*"product" + 0.017*"work" + 0.014*"day" + 0.014*"good" + 0.011*"use" + 0.011*"feel" + 0.011*"year" + 0.010*"supplement" + 0.010*"recommend"
INFO : topic #1 (0.191): 0.033*"good" + 0.023*"taste" + 0.022*"great" + 0.019*

DEBUG : 7000/7000 documents converged within 400 iterations
INFO : optimized alpha [0.2392199, 0.21064433, 0.19962873, 0.11766813]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.239): 0.019*"help" + 0.017*"product" + 0.016*"work" + 0.014*"day" + 0.014*"good" + 0.012*"year" + 0.011*"use" + 0.011*"feel" + 0.010*"recommend" + 0.010*"try"
INFO : topic #1 (0.211): 0.032*"good" + 0.027*"taste" + 0.022*"great" + 0.017*"like" + 0.017*"product" + 0.013*"easy" + 0.013*"vitamin" + 0.012*"use" + 0.011*"love" + 0.009*"supplement"
INFO : topic #2 (0.200): 0.072*"product" + 0.038*"great" + 0.034*"good" + 0.030*"use" + 0.018*"price" + 0.014*"love" + 0.014*"buy" + 0.013*"recommend" + 0.013*"skin" + 0.013*"work"
INFO : topic #3 (0.118): 0.024*"day" + 0.024*"work" + 0.017*"use" + 0.017*"pain" + 0.015*"product" + 0.014*"feel" + 0.013*"week" + 0.012*"start" + 0.011*"help" + 0.011*"try"
INFO : topic diff=0.132017, rho=0.173878
INFO : PR

INFO : topic #2 (0.217): 0.075*"product" + 0.038*"great" + 0.034*"good" + 0.027*"use" + 0.017*"price" + 0.014*"love" + 0.013*"buy" + 0.013*"recommend" + 0.013*"work" + 0.013*"skin"
INFO : topic #3 (0.126): 0.025*"work" + 0.023*"day" + 0.016*"feel" + 0.016*"product" + 0.016*"use" + 0.015*"pain" + 0.013*"week" + 0.012*"try" + 0.011*"energy" + 0.011*"start"
INFO : topic diff=0.143215, rho=0.173878
INFO : PROGRESS: pass 1, at document #203000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.23713008, 0.21013279, 0.22332871, 0.12718305]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.237): 0.019*"help" + 0.018*"product" + 0.016*"work" + 0.014*"good" + 0.013*"day" + 0.011*"use" + 0.011*"supplement" + 0.011*"feel" + 0.010*"year" + 0.010*"recommend"
INFO : topic #1 (0.210): 0.030*"good" + 0.027*"taste" + 0.021*"great" + 0.0

Trained LDA for 4 topics...


AttributeError: 'LdaModel' object has no attribute 'columns'

In [21]:
# error in the previous cell, so redoing some steps above:

# save_df_s3(lda, bucket_name, filepath='amazon_reviews/kk/lda_tier1_topics_4.pkl', filetype='pickle')
lda = load_df_s3(bucket_name, filepath='amazon_reviews/kk/lda_tier1_topics_4.pkl', filetype='pickle')

In [ ]:
# performance metric
cm = CoherenceModel(model=lda, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
coherenceList_umass.append(cm.get_coherence())
print('Calculated coherence score...: ', cm.get_coherence())

In [21]:
# visualization
vis = pyLDAvis.gensim.prepare(lda, bow_corpus, vocab_dictionary)

DEBUG : performing inference on a chunk of 217530 documents
DEBUG : 217516/217530 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [22]:
plot_fname = 'pyLDAvis_{}.html'.format(4)
pyLDAvis.save_html(vis, plot_fname)
print('Saved html file: ', plot_fname)

Saved html file:  pyLDAvis_4.html


In [74]:
pyLDAvis.display(vis)

In [71]:
lda.print_topics(4)

[(0,
  '0.020*"help" + 0.019*"product" + 0.015*"good" + 0.015*"supplement" + 0.015*"work" + 0.013*"feel" + 0.012*"day" + 0.011*"try" + 0.011*"use" + 0.010*"recommend"'),
 (1,
  '0.028*"good" + 0.025*"taste" + 0.021*"great" + 0.018*"like" + 0.016*"product" + 0.016*"easy" + 0.013*"supplement" + 0.012*"vitamin" + 0.011*"fish_oil" + 0.011*"love"'),
 (2,
  '0.078*"product" + 0.038*"great" + 0.030*"good" + 0.025*"use" + 0.019*"skin" + 0.014*"recommend" + 0.014*"love" + 0.013*"price" + 0.012*"work" + 0.011*"buy"'),
 (3,
  '0.023*"work" + 0.021*"feel" + 0.021*"day" + 0.019*"product" + 0.015*"pain" + 0.015*"week" + 0.015*"use" + 0.014*"try" + 0.012*"start" + 0.012*"energy"')]

In [10]:
reviews_lda = []

for doc_bow in tqdm(bow_corpus):
    reviews_lda.append(lda[doc_bow])

100%|██████████| 217530/217530 [03:09<00:00, 1147.45it/s]


In [11]:
reviews_lda[0]

[(0, 0.48668307), (1, 0.12851799), (2, 0.38285732)]

In [14]:
len(reviews_lda)

217530

In [33]:
save_df_s3(reviews_lda, bucket_name, filepath='amazon_reviews/kk/doc2topic_tier1.pkl', filetype='pickle')

In [5]:
# reviews_lda = load_df_s3(bucket_name, filepath='amazon_reviews/kk/doc2topic_tier1.pkl', filetype='pickle')

In [15]:
# reviews_lda is a list with one element (sublist) per document
# each sublist contains is a list of tuples of the format (topic_num, probability)
reviews_lda[0:3]

[[(0, 0.48668307), (1, 0.12851799), (2, 0.38285732)],
 [(0, 0.31406406), (2, 0.67967421)],
 [(0, 0.30542588), (1, 0.24088043), (2, 0.44061708), (3, 0.013076626)]]

In [22]:
# for each document, get the topic number with the highest probability
def get_first_order_topic(reviews_lda):
    first_order_topics = []
    for doc in reviews_lda:
        topic_max_prob = sorted(doc, key=lambda x: x[1], reverse=True)[0][0]
        first_order_topics.append(topic_max_prob)
    return first_order_topics

In [23]:
first_order_topics = get_first_order_topic(reviews_lda)

In [24]:
len(first_order_topics)

217530

In [29]:
Counter(first_order_topics).most_common()

[(0, 68797), (2, 59465), (1, 56018), (3, 33250)]

In [53]:
tokenized_reviews_temp = pd.DataFrame(unigram_sents_pos_df.groupby(('review_number'))['preprocessed_review'].sum())

In [54]:
tokenized_reviews_temp.head()

,preprocessed_review
review_number,
1,"[bioavailability, key, -PRON-, start, -PRON-, ..."
2,"[-PRON-, find, longer, -PRON-, wish, -PRON-, p..."
3,"[resveratrol, product, -PRON-, need, resveratr..."
4,"[kid, love, bracelet, -PRON-, buy, bracelet, -..."
5,"[pleasant, surprise, -PRON-, buy, week, -PRON-..."


In [65]:
# construct a df with the first order topics as a column
docs_first_order_lda = pd.DataFrame({'review_num': tokenized_reviews_temp.index.tolist(),
                                     'tokenized_reviews': tokenized_reviews, 
                                     'first_order_topics': first_order_topics,
                                    })

del tokenized_reviews_temp

In [83]:
docs_first_order_lda.head()

,first_order_topics,review_num,tokenized_reviews
0,0,1,"[bioavailability, key, start, parent, die, can..."
1,2,2,"[find, longer, wish, product, like, stay, mout..."
2,2,3,"[resveratrol, product, need, resveratrol, pola..."
3,3,4,"[kid, love, bracelet, buy, bracelet, ymca, kid..."
4,3,5,"[pleasant, surprise, buy, week, day, wear, bra..."


In [70]:
docs_first_order_lda.shape

(217530, 3)

In [6]:
# save_df_s3(docs_first_order_lda, bucket_name, filepath='amazon_reviews/kk/docs_first_order_lda_v1.pkl', filetype='pickle')
docs_first_order_lda = load_df_s3(bucket_name, filepath='amazon_reviews/kk/docs_first_order_lda_v1.pkl', filetype='pickle')

### Training LDA for Topic 0 Reviews

Note that topic # 1 in the LDA vis plot has index 0.

In [7]:
docs_topic_0 = docs_first_order_lda[docs_first_order_lda.first_order_topics == 0].tokenized_reviews.tolist()

In [8]:
print(docs_topic_0[0])

['bioavailability', 'key', 'start', 'parent', 'die', 'cancer', 'suppose', 'enhance', 'immune_system', 'story', 'resveratrol', 'minutes', 'incredibly', 'inspiring', 'research', 'internet', 'indicate', 'resveratrol', 'lozenge', 'form', 'preferable', 'stomach', 'acid', 'break', 'ez', 'melt', 'formula', 'recommend', 'review', 'ok', 'dissolve', 'mouth', 'quickly', 'lozenge', 'formula', 'dissolve', 'slowly', 'preferable', 'accord', 'research', 'great', 'side_effect', 'cold', 'sore_throat', 'product', 'soon', 'start', 'start', 'come', 'cold', 'usual', 'symptom', 'anticipate', 'sick', 'day', 'usual', 'pattern', 'day', 'sick', 'anticipate', 'taking', 'reason', 'come', 'product', 'cold', 'sore_throat', 'great', 'recommend', 'know', 'gift', 'family', 'product']


In [9]:
docs_topic_1 = docs_first_order_lda[docs_first_order_lda.first_order_topics == 1].tokenized_reviews.tolist()

In [10]:
docs_topic_2 = docs_first_order_lda[docs_first_order_lda.first_order_topics == 2].tokenized_reviews.tolist()

#### Topic 0

In [97]:
# Topic 0
vocab_dictionary_0 = Dictionary(docs_topic_0)
once_ids = [tokenid for tokenid, docfreq in iteritems(vocab_dictionary_0.dfs) if docfreq == 1]
vocab_dictionary_0.filter_tokens(bad_ids=once_ids)  # remove words that appear only once
vocab_dictionary_0.filter_extremes(no_below=10, no_above=1.0)
vocab_dictionary_0.compactify() 
save_df_s3(vocab_dictionary_0, bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_topic_0.dict', filetype='pickle')

In [11]:
vocab_dictionary_0 = load_df_s3(bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_topic_0.dict', filetype='pickle')

In [12]:
print(vocab_dictionary_0)

Dictionary(8233 unique tokens: ['accord', 'acid', 'anticipate', 'bioavailability', 'break']...)


In [12]:
bow_corpus_0 = [vocab_dictionary_0.doc2bow(review) for review in docs_topic_0]

In [13]:
%%time

# Set training parameters.
num_topics_list = [3, 4, 5]
chunksize = 7000    # number of docs processed at a time
passes = 2
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = None           # symmetric prior
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda_0 = LdaModel(bow_corpus_0, num_topics=num_topics, id2word=vocab_dictionary_0, chunksize=chunksize, 
                     passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    save_df_s3(lda_0, bucket_name, filepath='amazon_reviews/kk/lda_tier2_topic0_{}.pkl'.format(num_topics), filetype='pickle')
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda_0, corpus=bow_corpus_0, dictionary=vocab_dictionary_0, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda_0, bow_corpus_0, vocab_dictionary_0)
    plot_fname = 'pyLDAvis_tier2_topic0_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/3 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
INFO : using symmetric eta at 0.3333333333333333
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 3 topics, 2 passes over the supplied corpus of 68797 documents, updating model once every 7000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #7000/68797
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6431/7000 documents converged within 400 iterations
INFO : optimized alpha [0.13363323, 0.064715683, 0.21139389]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 68797 documents
INFO : topic #0 (0.134): 0.021*"good" + 0.020*"product" + 0.018*"work" + 0.013*"day" + 0.013*"help" + 0.012*"feel" + 0.012*"great" + 0.011*"use" + 0.011*"recommend" + 0.010*"year"
INFO : topic #1 (0.065): 0.01

INFO : topic #1 (0.075): 0.020*"work" + 0.019*"product" + 0.016*"day" + 0.016*"help" + 0.015*"use" + 0.013*"year" + 0.013*"sleep" + 0.011*"try" + 0.011*"time" + 0.010*"good"
INFO : topic #2 (0.135): 0.026*"product" + 0.019*"good" + 0.016*"use" + 0.015*"help" + 0.014*"supplement" + 0.013*"great" + 0.011*"work" + 0.009*"recommend" + 0.009*"day" + 0.009*"year"
INFO : topic diff=0.277873, rho=0.333333
INFO : PROGRESS: pass 0, at document #68797/68797
DEBUG : performing inference on a chunk of 5797 documents
DEBUG : 5792/5797 documents converged within 400 iterations
INFO : optimized alpha [0.11213784, 0.074389808, 0.13245399]
DEBUG : updating topics
INFO : merging changes from 5797 documents into a model of 68797 documents
INFO : topic #0 (0.112): 0.026*"product" + 0.020*"good" + 0.016*"feel" + 0.014*"work" + 0.014*"help" + 0.013*"day" + 0.012*"probiotic" + 0.011*"try" + 0.011*"use" + 0.010*"great"
INFO : topic #1 (0.074): 0.020*"work" + 0.019*"product" + 0.016*"help" + 0.016*"day" + 0.015

INFO : topic #2 (0.117): 0.024*"product" + 0.019*"good" + 0.015*"supplement" + 0.014*"use" + 0.012*"help" + 0.011*"great" + 0.010*"work" + 0.010*"recommend" + 0.009*"year" + 0.009*"doctor"
INFO : topic diff=0.206205, rho=0.290765
INFO : PROGRESS: pass 1, at document #63000/68797
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.10642471, 0.087443143, 0.11678594]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 68797 documents
INFO : topic #0 (0.106): 0.027*"product" + 0.020*"good" + 0.016*"work" + 0.016*"help" + 0.014*"feel" + 0.013*"day" + 0.013*"use" + 0.012*"try" + 0.012*"great" + 0.011*"probiotic"
INFO : topic #1 (0.087): 0.021*"work" + 0.019*"product" + 0.017*"help" + 0.017*"day" + 0.016*"use" + 0.013*"year" + 0.013*"sleep" + 0.012*"good" + 0.011*"try" + 0.011*"time"
INFO : topic #2 (0.117): 0.024*"product" + 0.019*"good" + 0.016*"supplement" + 0.014*"use

Trained LDA for 3 topics...


INFO : CorpusAccumulator accumulated stats from 17000 documents
INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated stats from 30000 documents
INFO : CorpusAccumulator accumulated stats from 31000 documents
INFO : CorpusAccumulator accumulated sta

...

Trained LDA for 5 topics...


INFO : CorpusAccumulator accumulated stats from 16000 documents
INFO : CorpusAccumulator accumulated stats from 17000 documents
INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated stats from 30000 documents
INFO : CorpusAccumulator accumulated sta

Calculated coherence score...:  -1.90241640371


DEBUG : performing inference on a chunk of 68797 documents
DEBUG : 68783/68797 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 3/3 [11:34<00:00, 231.41s/it]

Saved html file:  pyLDAvis_tier2_topic0_5.html
CPU times: user 11min 32s, sys: 2.83 s, total: 11min 34s
Wall time: 11min 34s


In [23]:
coherenceList_umass

[-1.8111989778566919, -1.8558095703462949, -1.9024164037097104]

#### Topic 1

In [27]:
# Topic 1
vocab_dictionary_1 = Dictionary(docs_topic_1)
once_ids = [tokenid for tokenid, docfreq in iteritems(vocab_dictionary_1.dfs) if docfreq == 1]
vocab_dictionary_1.filter_tokens(bad_ids=once_ids)  # remove words that appear only once
vocab_dictionary_1.filter_extremes(no_below=10, no_above=1.0)
vocab_dictionary_1.compactify()
save_df_s3(vocab_dictionary_1, bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_topic1.dict', filetype='pickle')

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(16478 unique tokens: ['bracelet', 'color', 'elegant', 'forget', 'light']...)
INFO : adding document #20000 to Dictionary(25318 unique tokens: ['bracelet', 'color', 'elegant', 'forget', 'light']...)
INFO : adding document #30000 to Dictionary(31718 unique tokens: ['bracelet', 'color', 'elegant', 'forget', 'light']...)
INFO : adding document #40000 to Dictionary(37994 unique tokens: ['bracelet', 'color', 'elegant', 'forget', 'light']...)
INFO : adding document #50000 to Dictionary(44538 unique tokens: ['bracelet', 'color', 'elegant', 'forget', 'light']...)
INFO : built Dictionary(48406 unique tokens: ['bracelet', 'color', 'elegant', 'forget', 'light']...) from 56018 documents (total 1467096 corpus positions)
DEBUG : rebuilding dictionary, shrinking gaps
INFO : discarding 11177 tokens: [('bracelet', 3), ('elegant', 6), ('mon', 5), ('om', 7), ('survival', 8), ('brave', 8), ('motrin', 5)

In [ ]:
# vocab_dictionary_1 = load_df_s3(bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_topic1.dict', filetype='pickle')

In [28]:
print(vocab_dictionary_1)

Dictionary(5974 unique tokens: ['color', 'forget', 'light', 'love', 'nice']...)


In [29]:
bow_corpus_1 = [vocab_dictionary_1.doc2bow(review) for review in docs_topic_1]

In [30]:
%%time

# Set training parameters.
num_topics_list = [3, 4, 5]
chunksize = 7000    # number of docs processed at a time
passes = 2
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = None           # symmetric prior
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda_1 = LdaModel(bow_corpus_1, num_topics=num_topics, id2word=vocab_dictionary_1, chunksize=chunksize, 
                     passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    save_df_s3(lda_1, bucket_name, filepath='amazon_reviews/kk/lda_tier2_topic1_{}.pkl'.format(num_topics), filetype='pickle')
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda_1, corpus=bow_corpus_1, dictionary=vocab_dictionary_1, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda_1, bow_corpus_1, vocab_dictionary_1)
    plot_fname = 'pyLDAvis_tier2_topic1_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/3 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
INFO : using symmetric eta at 0.3333333333333333
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 3 topics, 2 passes over the supplied corpus of 56018 documents, updating model once every 7000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #7000/56018
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6599/7000 documents converged within 400 iterations
INFO : optimized alpha [0.11540811, 0.16973124, 0.15536271]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 56018 documents
INFO : topic #0 (0.115): 0.023*"good" + 0.017*"product" + 0.016*"use" + 0.014*"taste" + 0.014*"day" + 0.013*"supplement" + 0.011*"like" + 0.011*"great" + 0.009*"vitamin" + 0.008*"pill"
INFO : topic #1 (0.170): 

INFO : topic #1 (0.107): 0.041*"taste" + 0.026*"great" + 0.023*"product" + 0.023*"good" + 0.022*"like" + 0.020*"use" + 0.016*"love" + 0.012*"flavor" + 0.011*"try" + 0.010*"water"
INFO : topic #2 (0.141): 0.031*"taste" + 0.030*"product" + 0.030*"good" + 0.030*"fish_oil" + 0.024*"fish" + 0.023*"great" + 0.020*"like" + 0.020*"pill" + 0.019*"try" + 0.019*"fishy"
INFO : topic diff=0.377676, rho=0.333333
INFO : PROGRESS: pass 1, at document #7000/56018
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6997/7000 documents converged within 400 iterations
INFO : optimized alpha [0.082386926, 0.11447696, 0.12582971]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 56018 documents
INFO : topic #0 (0.082): 0.032*"vitamin" + 0.023*"supplement" + 0.017*"good" + 0.016*"product" + 0.015*"tablet" + 0.014*"great" + 0.013*"use" + 0.012*"need" + 0.010*"calcium" + 0.010*"day"
INFO : topic #1 (0.114): 0.035*"taste" + 0.025*"great" + 0.024*"good" + 0.022*"pro

INFO : topic #1 (0.109): 0.038*"taste" + 0.024*"great" + 0.023*"good" + 0.022*"product" + 0.021*"like" + 0.020*"use" + 0.014*"love" + 0.013*"flavor" + 0.011*"try" + 0.010*"water"
INFO : topic #2 (0.116): 0.037*"taste" + 0.034*"fish_oil" + 0.031*"good" + 0.031*"product" + 0.027*"fish" + 0.024*"pill" + 0.023*"try" + 0.023*"like" + 0.023*"great" + 0.022*"love"
INFO : topic diff=0.356704, rho=0.316187
DEBUG : Setting topics to those of the model: LdaModel(num_terms=5974, num_topics=3, decay=0.5, chunksize=7000)
INFO : CorpusAccumulator accumulated stats from 1000 documents
INFO : CorpusAccumulator accumulated stats from 2000 documents
INFO : CorpusAccumulator accumulated stats from 3000 documents
INFO : CorpusAccumulator accumulated stats from 4000 documents
INFO : CorpusAccumulator accumulated stats from 5000 documents
INFO : CorpusAccumulator accumulated stats from 6000 documents
INFO : CorpusAccumulator accumulated stats from 7000 documents
INFO : CorpusAccumulator accumulated stats fro

Trained LDA for 3 topics...


INFO : CorpusAccumulator accumulated stats from 16000 documents
INFO : CorpusAccumulator accumulated stats from 17000 documents
INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated stats from 30000 documents
INFO : CorpusAccumulator accumulated sta

...

Trained LDA for 5 topics...


INFO : CorpusAccumulator accumulated stats from 17000 documents
INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated stats from 30000 documents
INFO : CorpusAccumulator accumulated stats from 31000 documents
INFO : CorpusAccumulator accumulated sta

Calculated coherence score...:  -2.373443323


DEBUG : performing inference on a chunk of 56018 documents
DEBUG : 56017/56018 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 3/3 [08:21<00:00, 167.04s/it]

Saved html file:  pyLDAvis_tier2_topic1_5.html
CPU times: user 8min 18s, sys: 2.59 s, total: 8min 21s
Wall time: 8min 21s


#### Topic 2

In [31]:
# Topic 2
vocab_dictionary_2 = Dictionary(docs_topic_2)
once_ids = [tokenid for tokenid, docfreq in iteritems(vocab_dictionary_2.dfs) if docfreq == 1]
vocab_dictionary_2.filter_tokens(bad_ids=once_ids)  # remove words that appear only once
vocab_dictionary_2.filter_extremes(no_below=10, no_above=1.0)
vocab_dictionary_2.compactify()
save_df_s3(vocab_dictionary_2, bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_topic2.dict', filetype='pickle')

INFO : adding document #0 to Dictionary(0 unique tokens: [])
INFO : adding document #10000 to Dictionary(13311 unique tokens: ['anymore', 'believe', 'claim', 'company', 'contact']...)
INFO : adding document #20000 to Dictionary(20414 unique tokens: ['anymore', 'believe', 'claim', 'company', 'contact']...)
INFO : adding document #30000 to Dictionary(26038 unique tokens: ['anymore', 'believe', 'claim', 'company', 'contact']...)
INFO : adding document #40000 to Dictionary(31383 unique tokens: ['anymore', 'believe', 'claim', 'company', 'contact']...)
INFO : adding document #50000 to Dictionary(36542 unique tokens: ['anymore', 'believe', 'claim', 'company', 'contact']...)
INFO : built Dictionary(41144 unique tokens: ['anymore', 'believe', 'claim', 'company', 'contact']...) from 59465 documents (total 1170796 corpus positions)
DEBUG : rebuilding dictionary, shrinking gaps
INFO : discarding 9786 tokens: [('introduction', 7), ('minutes', 4), ('someplace', 2), ('financial', 6), ('insoluble', 2)

In [ ]:
# vocab_dictionary_2 = load_df_s3(bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_topic2.dict', filetype='pickle')

In [32]:
print(vocab_dictionary_2)

Dictionary(4936 unique tokens: ['anymore', 'believe', 'claim', 'company', 'contact']...)


In [33]:
bow_corpus_2 = [vocab_dictionary_2.doc2bow(review) for review in docs_topic_2]

In [34]:
%%time

# Set training parameters.
num_topics_list = [3, 4, 5]
chunksize = 7000    # number of docs processed at a time
passes = 2
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = None           # symmetric prior
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda_2 = LdaModel(bow_corpus_2, num_topics=num_topics, id2word=vocab_dictionary_2, chunksize=chunksize, 
                     passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, random_state=42)
    save_df_s3(lda_2, bucket_name, filepath='amazon_reviews/kk/lda_tier2_topic2_{}.pkl'.format(num_topics), filetype='pickle')
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda_2, corpus=bow_corpus_2, dictionary=vocab_dictionary_2, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda_2, bow_corpus_2, vocab_dictionary_2)
    plot_fname = 'pyLDAvis_tier2_topic2_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/3 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
INFO : using symmetric eta at 0.3333333333333333
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 3 topics, 2 passes over the supplied corpus of 59465 documents, updating model once every 7000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #7000/59465
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6759/7000 documents converged within 400 iterations
INFO : optimized alpha [0.19838814, 0.21123424, 0.096921355]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 59465 documents
INFO : topic #0 (0.198): 0.056*"product" + 0.032*"great" + 0.030*"use" + 0.027*"good" + 0.017*"skin" + 0.017*"work" + 0.014*"recommend" + 0.012*"buy" + 0.011*"time" + 0.011*"order"
INFO : topic #1 (0.211): 0.0

INFO : topic #1 (0.162): 0.075*"product" + 0.042*"great" + 0.040*"good" + 0.020*"price" + 0.017*"use" + 0.014*"buy" + 0.014*"love" + 0.012*"recommend" + 0.012*"quality" + 0.010*"supplement"
INFO : topic #2 (0.063): 0.049*"hair" + 0.031*"skin" + 0.022*"use" + 0.022*"product" + 0.020*"nail" + 0.014*"grow" + 0.014*"great" + 0.014*"notice" + 0.014*"look" + 0.013*"love"
INFO : topic diff=0.358805, rho=0.333333
INFO : PROGRESS: pass 1, at document #7000/59465
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6997/7000 documents converged within 400 iterations
INFO : optimized alpha [0.12075604, 0.16356596, 0.062308125]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 59465 documents
INFO : topic #0 (0.121): 0.050*"product" + 0.032*"use" + 0.028*"skin" + 0.022*"great" + 0.018*"good" + 0.016*"work" + 0.013*"recommend" + 0.012*"try" + 0.011*"feel" + 0.010*"day"
INFO : topic #1 (0.164): 0.073*"product" + 0.044*"great" + 0.041*"good" + 0.022*"pric

INFO : topic #2 (0.054): 0.063*"hair" + 0.025*"nail" + 0.022*"use" + 0.020*"skin" + 0.018*"product" + 0.018*"grow" + 0.013*"notice" + 0.013*"love" + 0.012*"strong" + 0.012*"month"
INFO : topic diff=0.302513, rho=0.308680
DEBUG : Setting topics to those of the model: LdaModel(num_terms=4936, num_topics=3, decay=0.5, chunksize=7000)
INFO : CorpusAccumulator accumulated stats from 1000 documents
INFO : CorpusAccumulator accumulated stats from 2000 documents
INFO : CorpusAccumulator accumulated stats from 3000 documents
INFO : CorpusAccumulator accumulated stats from 4000 documents
INFO : CorpusAccumulator accumulated stats from 5000 documents
INFO : CorpusAccumulator accumulated stats from 6000 documents
INFO : CorpusAccumulator accumulated stats from 7000 documents
INFO : CorpusAccumulator accumulated stats from 8000 documents
INFO : CorpusAccumulator accumulated stats from 9000 documents
INFO : CorpusAccumulator accumulated stats from 10000 documents
INFO : CorpusAccumulator accumulated

Trained LDA for 3 topics...


INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated stats from 30000 documents
INFO : CorpusAccumulator accumulated stats from 31000 documents
INFO : CorpusAccumulator accumulated stats from 32000 documents
INFO : CorpusAccumulator accumulated stats from 33000 documents
INFO : CorpusAccumulator accumulated stats from 34000 documents
INFO : CorpusAccumulator accumulated sta

...

Trained LDA for 5 topics...


INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated stats from 30000 documents
INFO : CorpusAccumulator accumulated stats from 31000 documents
INFO : CorpusAccumulator accumulated stats from 32000 documents
INFO : CorpusAccumulator accumulated stats from 33000 documents
INFO : CorpusAccumulator accumulated sta

Calculated coherence score...:  -2.11043852988


DEBUG : performing inference on a chunk of 59465 documents
DEBUG : 59464/59465 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 3/3 [08:06<00:00, 162.24s/it]

Saved html file:  pyLDAvis_tier2_topic2_5.html
CPU times: user 8min 4s, sys: 2.52 s, total: 8min 6s
Wall time: 8min 6s


## Seeded LDA using Gensim

`eta` can be a scalar for a symmetric prior over topic/word distributions, or a matrix of shape **`num_topics` x `num_words`**, which can be used to impose asymmetric priors over the word distribution on a per-topic basis. This may be useful if you want to seed certain topics with particular words by boosting the priors for those words.

In [6]:
vocab_dictionary = load_df_s3(bucket_name, filepath='amazon_reviews/kk/vocab_dictionary_v3.dict', filetype='pickle')
print(vocab_dictionary)

Dictionary(13850 unique tokens: ['accord', 'acid', 'anticipate', 'bioavailability', 'break']...)


In [7]:
tokenized_reviews = load_df_s3(bucket_name, filepath='amazon_reviews/kk/tokenized_reviews_v1.pkl', filetype='pickle')
# bag-of-words representation of the corpus/ doc-term matrix
bow_corpus = [vocab_dictionary.doc2bow(review) for review in tokenized_reviews]

In [8]:
# topics and associated words
efficacy = ['quality', 'work', 'help', 'result', 'notice', 'help', 'effect', 'effective', 'outcome', 'high_quality']
cost = ['price', 'cost', 'money', 'expensive', 'cheap', 'value', 'worth']
seller = ['seller', 'shipping', 'service', 'ship', 'customer', 'customer_service', 'timely', 'company', 'order', 'refund', 'return']

In [9]:
# convert topics' words to ids
efficacy_id = [vocab_dictionary.token2id[word] for word in efficacy]
cost_id = [vocab_dictionary.token2id[word] for word in cost]
seller_id = [vocab_dictionary.token2id[word] for word in seller]

In [10]:
efficacy_id

[767, 189, 105, 297, 182, 105, 453, 832, 558, 1881]

In [11]:
topics = [efficacy_id, cost_id, seller_id]
num_topics = len(topics)

# initialize eta: matrix of shape num_topics x num_words
eta = np.ones((num_topics, len(vocab_dictionary))) * (1.0/num_topics)
baseline_prob = eta[0, 0]

boosted_prob = 0.9

for i in range(num_topics):
    
    topic_ids = topics[i]   # word ids related to topic i
    
    for seeded_word_id in topics[i]:
        # boost probability of selected word in current topic
        eta[i, seeded_word_id] = boosted_prob
        # distribute the leftover probability over the remaining topics
        leftover_topic_rownums = [x for x in range(num_topics) if x != i]
        eta[leftover_topic_rownums, seeded_word_id] = ((1 - boosted_prob) / (num_topics - 1))

In [12]:
eta[0, vocab_dictionary.token2id['quality']]

0.90000000000000002

In [13]:
eta[1, vocab_dictionary.token2id['quality']]

0.049999999999999989

In [14]:
eta[2, vocab_dictionary.token2id['quality']]

0.049999999999999989

In [15]:
eta

array([[ 0.33333333,  0.33333333,  0.33333333, ...,  0.33333333,
         0.33333333,  0.33333333],
       [ 0.33333333,  0.33333333,  0.33333333, ...,  0.33333333,
         0.33333333,  0.33333333],
       [ 0.33333333,  0.33333333,  0.33333333, ...,  0.33333333,
         0.33333333,  0.33333333]])

In [16]:
eta.shape

(3, 13850)

In [17]:
%%time

# Set training parameters.
num_topics_list = [3]
chunksize = 7000    # number of docs processed at a time
passes = 3
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = eta           # asymmetric prior (seeded)
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda_seeded = LdaModel(bow_corpus, num_topics=num_topics, id2word=vocab_dictionary, chunksize=chunksize, 
                          passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, 
                          random_state=42)
    save_df_s3(lda_seeded, bucket_name, filepath='amazon_reviews/kk/lda_seeded_{}.pkl'.format(num_topics), 
               filetype='pickle')
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda_seeded, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda_seeded, bow_corpus, vocab_dictionary)
    plot_fname = 'pyLDAvis_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/1 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 3 topics, 3 passes over the supplied corpus of 217530 documents, updating model once every 7000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #7000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6555/7000 documents converged within 400 iterations
INFO : optimized alpha [0.10288081, 0.13132362, 0.19620216]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.103): 0.025*"pedometer" + 0.023*"day" + 0.022*"step" + 0.018*"use" + 0.016*"great" + 0.016*"good" + 0.014*"like" + 0.014*"work" + 0.011*"product" + 0.011*"walk"
INFO : topic #1 (0.131): 0.030*"pedometer" + 0.022*"great" + 0.022*"use" + 0

INFO : topic #2 (0.127): 0.046*"product" + 0.029*"use" + 0.024*"good" + 0.022*"great" + 0.018*"work" + 0.012*"skin" + 0.012*"love" + 0.011*"year" + 0.010*"help" + 0.010*"recommend"
INFO : topic diff=0.360838, rho=0.333333
INFO : PROGRESS: pass 0, at document #70000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.14829192, 0.14594503, 0.12849703]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.148): 0.018*"day" + 0.017*"work" + 0.016*"help" + 0.014*"product" + 0.013*"good" + 0.011*"feel" + 0.011*"use" + 0.010*"year" + 0.010*"try" + 0.009*"start"
INFO : topic #1 (0.146): 0.034*"product" + 0.032*"good" + 0.027*"great" + 0.017*"use" + 0.017*"taste" + 0.012*"like" + 0.012*"price" + 0.009*"find" + 0.009*"love" + 0.009*"work"
INFO : topic #2 (0.128): 0.047*"product" + 0.029*"use" + 0.023*"good" + 0.022*"great" + 0.018*"wo

INFO : topic #2 (0.121): 0.047*"product" + 0.030*"use" + 0.023*"great" + 0.020*"good" + 0.018*"work" + 0.013*"skin" + 0.012*"love" + 0.010*"recommend" + 0.010*"help" + 0.010*"year"
INFO : topic diff=0.231784, rho=0.235702
INFO : PROGRESS: pass 0, at document #133000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6997/7000 documents converged within 400 iterations
INFO : optimized alpha [0.1626647, 0.18799931, 0.12085097]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.163): 0.018*"day" + 0.018*"work" + 0.017*"help" + 0.015*"product" + 0.012*"good" + 0.011*"feel" + 0.011*"use" + 0.010*"year" + 0.010*"try" + 0.009*"start"
INFO : topic #1 (0.188): 0.036*"good" + 0.031*"product" + 0.027*"great" + 0.021*"taste" + 0.014*"use" + 0.014*"like" + 0.013*"price" + 0.011*"vitamin" + 0.010*"love" + 0.009*"find"
INFO : topic #2 (0.121): 0.047*"product" + 0.031*"use" + 0.024*"great" + 0.020*"good" + 0.018*

INFO : topic diff=0.195451, rho=0.192450
INFO : PROGRESS: pass 0, at document #196000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6997/7000 documents converged within 400 iterations
INFO : optimized alpha [0.18187711, 0.19608344, 0.12382407]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.182): 0.018*"work" + 0.017*"product" + 0.017*"day" + 0.016*"help" + 0.013*"good" + 0.013*"feel" + 0.011*"use" + 0.010*"try" + 0.009*"year" + 0.009*"supplement"
INFO : topic #1 (0.196): 0.034*"good" + 0.032*"product" + 0.027*"great" + 0.020*"taste" + 0.014*"like" + 0.013*"use" + 0.012*"price" + 0.011*"vitamin" + 0.010*"easy" + 0.010*"love"
INFO : topic #2 (0.124): 0.052*"product" + 0.030*"use" + 0.026*"great" + 0.020*"good" + 0.018*"work" + 0.014*"skin" + 0.013*"love" + 0.013*"hair" + 0.010*"recommend" + 0.010*"feel"
INFO : topic diff=0.194153, rho=0.188982
INFO : PROGRESS: pass 0, at document #203000/21

INFO : topic diff=0.179011, rho=0.173878
INFO : PROGRESS: pass 1, at document #35000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6997/7000 documents converged within 400 iterations
INFO : optimized alpha [0.19869582, 0.18542501, 0.131529]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.199): 0.018*"work" + 0.018*"help" + 0.017*"day" + 0.017*"product" + 0.013*"good" + 0.013*"feel" + 0.012*"use" + 0.010*"try" + 0.009*"year" + 0.009*"recommend"
INFO : topic #1 (0.185): 0.033*"good" + 0.031*"product" + 0.027*"great" + 0.018*"taste" + 0.014*"like" + 0.014*"use" + 0.013*"price" + 0.012*"easy" + 0.010*"vitamin" + 0.010*"love"
INFO : topic #2 (0.132): 0.049*"product" + 0.031*"use" + 0.026*"great" + 0.019*"good" + 0.018*"work" + 0.018*"skin" + 0.015*"love" + 0.010*"recommend" + 0.010*"hair" + 0.010*"feel"
INFO : topic diff=0.170773, rho=0.173878
INFO : PROGRESS: pass 1, at document #42000/217530


...

INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated stats from 29000 documents
INFO : CorpusAccumulator accumulated stats from 30000 documents
INFO : CorpusAccumulator accumulated stats from 31000 documents
INFO : CorpusAccumulator accumulated stats from 32000 documents
INFO : CorpusAccumulator accumulated sta

INFO : CorpusAccumulator accumulated stats from 146000 documents
INFO : CorpusAccumulator accumulated stats from 147000 documents
INFO : CorpusAccumulator accumulated stats from 148000 documents
INFO : CorpusAccumulator accumulated stats from 149000 documents
INFO : CorpusAccumulator accumulated stats from 150000 documents
INFO : CorpusAccumulator accumulated stats from 151000 documents
INFO : CorpusAccumulator accumulated stats from 152000 documents
INFO : CorpusAccumulator accumulated stats from 153000 documents
INFO : CorpusAccumulator accumulated stats from 154000 documents
INFO : CorpusAccumulator accumulated stats from 155000 documents
INFO : CorpusAccumulator accumulated stats from 156000 documents
INFO : CorpusAccumulator accumulated stats from 157000 documents
INFO : CorpusAccumulator accumulated stats from 158000 documents
INFO : CorpusAccumulator accumulated stats from 159000 documents
INFO : CorpusAccumulator accumulated stats from 160000 documents
INFO : CorpusAccumulator 

Calculated coherence score...:  -2.06992543002


DEBUG : performing inference on a chunk of 217530 documents
DEBUG : 217515/217530 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 1/1 [11:03<00:00, 663.41s/it]

Saved html file:  pyLDAvis_3.html
CPU times: user 11min 3s, sys: 1.58 s, total: 11min 4s
Wall time: 11min 3s


In [18]:
from gensim.models import Word2Vec

In [21]:
review2vec = Word2Vec(tokenized_reviews, sg=1)

INFO : collecting all words and their counts
INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
INFO : PROGRESS: at sentence #10000, processed 270420 words, keeping 17012 word types
INFO : PROGRESS: at sentence #20000, processed 538225 words, keeping 27237 word types
INFO : PROGRESS: at sentence #30000, processed 794667 words, keeping 35107 word types
INFO : PROGRESS: at sentence #40000, processed 1060869 words, keeping 42511 word types
INFO : PROGRESS: at sentence #50000, processed 1334096 words, keeping 49488 word types
INFO : PROGRESS: at sentence #60000, processed 1599525 words, keeping 55872 word types
INFO : PROGRESS: at sentence #70000, processed 1871260 words, keeping 62397 word types
INFO : PROGRESS: at sentence #80000, processed 2136438 words, keeping 67941 word types
INFO : PROGRESS: at sentence #90000, processed 2416725 words, keeping 73936 word types
INFO : PROGRESS: at sentence #100000, processed 2670691 words, keeping 79120 word types
INFO : PROGRES

INFO : EPOCH 4 - PROGRESS: at 70.72% examples, 368500 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 4 - PROGRESS: at 79.35% examples, 371133 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 4 - PROGRESS: at 87.09% examples, 370536 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 4 - PROGRESS: at 94.53% examples, 370537 words/s, in_qsize 5, out_qsize 0
DEBUG : job loop exiting, total 579 jobs
DEBUG : worker exiting, processed 193 jobs
INFO : worker thread finished; awaiting finish of 2 more threads
DEBUG : worker exiting, processed 194 jobs
INFO : worker thread finished; awaiting finish of 1 more threads
DEBUG : worker exiting, processed 192 jobs
INFO : worker thread finished; awaiting finish of 0 more threads
INFO : EPOCH - 4 : training on 5767227 raw words (4819483 effective words) took 13.0s, 371372 effective words/s
INFO : EPOCH 5 - PROGRESS: at 7.32% examples, 347254 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 5 - PROGRESS: at 15.19% examples, 359726 words/s, in_qsize 5, out_qsize 0
IN

In [24]:
review2vec.train(tokenized_reviews, total_examples=len(tokenized_reviews), epochs=5)

INFO : training model with 3 workers on 22056 vocabulary and 100 features, using sg=1 hs=0 sample=0.001 negative=5 window=5
INFO : EPOCH 1 - PROGRESS: at 7.32% examples, 351169 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 15.19% examples, 361941 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 23.03% examples, 365989 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 30.44% examples, 362596 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 38.29% examples, 365926 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 45.66% examples, 364522 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 53.93% examples, 368171 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 62.45% examples, 369269 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 70.36% examples, 368644 words/s, in_qsize 5, out_qsize 0
INFO : EPOCH 1 - PROGRESS: at 78.56% examples, 368899 words/s, in_qsize 5, out_qsize 0
INFO : 

(24096320, 28836135)

In [25]:
save_df_s3(review2vec, bucket_name, filepath='amazon_reviews/kk/reviews2vec.pkl', filetype='pickle')

In [57]:
# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count) for term, voc in review2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda x: -x[2])

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)
# print(ordered_terms)
# create a DataFrame with the food2vec vectors as data,
# and the terms as row labels
word_vectors = pd.DataFrame(review2vec.wv.syn0[term_indices, :], index=ordered_terms)

word_vectors

/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  del sys.path[0]


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
product,-0.074639,0.354219,-0.063841,0.089670,0.124870,-0.068005,-0.013207,0.077418,-0.072039,0.329119,...,-0.090658,0.023709,0.283028,0.135874,-0.020405,-0.348821,0.081809,-0.030812,-0.143888,0.085219
good,0.060326,0.231529,-0.074902,0.085871,0.125798,-0.210218,-0.254523,0.022960,-0.008771,0.107868,...,-0.112741,0.317170,0.317087,0.147866,0.027891,-0.198109,-0.161131,0.182401,-0.188499,0.284355
great,-0.023763,0.227623,0.004929,-0.032541,0.218707,-0.283889,-0.056582,-0.082675,-0.077025,-0.132305,...,-0.172068,0.273706,0.362132,0.212433,0.006429,-0.145674,-0.378388,0.332170,-0.047167,0.048488
use,0.155800,0.033982,0.285413,-0.048341,0.064590,-0.048613,0.025682,-0.090392,0.168781,-0.154475,...,-0.163690,0.172842,0.176484,0.065678,0.064181,-0.306245,-0.179612,0.168094,-0.231233,0.250193
work,-0.131723,0.010586,0.025562,-0.039717,0.244143,-0.032734,-0.207010,0.207864,0.275477,-0.119843,...,-0.228646,0.007168,0.179578,0.260665,0.066437,-0.443599,0.041221,0.112038,-0.188126,0.271196
day,-0.019957,-0.195679,0.167550,-0.076098,0.005321,-0.005643,0.103599,0.043114,-0.069537,0.089945,...,-0.292399,0.245713,-0.204933,0.163944,-0.119181,-0.182083,0.054396,0.083126,-0.313821,0.242774
help,0.070785,-0.172536,-0.329890,0.100656,0.269875,-0.180524,-0.067899,0.083343,0.031652,-0.123470,...,0.213810,0.254655,0.104915,0.258943,-0.088309,-0.146966,-0.194560,0.276072,-0.179112,0.399372
like,0.073624,0.001295,0.156824,0.017900,-0.129977,-0.087569,-0.062197,0.110463,-0.046887,0.224330,...,-0.107319,0.047285,0.165261,0.395306,0.015415,-0.164832,-0.232992,0.114996,0.029391,0.416531
recommend,0.294066,0.640802,0.152148,-0.030043,0.277109,-0.604638,-0.013796,0.093345,0.078040,0.171017,...,-0.179922,0.101642,0.061390,0.195905,-0.165352,-0.126587,-0.250627,0.189282,-0.168884,0.025101
try,-0.116470,0.008058,0.083074,0.211078,0.020380,-0.031157,0.010769,-0.054648,0.059417,-0.110198,...,-0.038766,0.008201,-0.047384,0.411964,-0.135511,-0.198599,-0.211693,0.150326,-0.428607,0.217542


In [88]:
def get_related_terms(token, topn=10):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """
    similar_words = []
    for word, similarity in review2vec.wv.most_similar(positive=[token], topn=topn):
        similar_words.append(word)
        print('{:20} {}'.format(word, round(similarity, 3)))
    return similar_words

In [76]:
efficacy = get_related_terms('efficacy', topn=3)

effectiveness        0.781
usefulness           0.72
validity             0.673


/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [86]:
efficacy += get_related_terms('quality', topn=10)

high_quality         0.857
purity               0.706
value                0.702
quality!.            0.701
quaility             0.685
products!!.          0.685
unsurpassed          0.68
quality!!.           0.68
excellence!.         0.679
reputation           0.676


/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


In [174]:
efficacy = []
cost = []
service = []

In [175]:
efficacy += get_related_terms('effect', topn=10)

negative_side_effect 0.697
side_effect          0.692
positive_effect      0.683
side_affect          0.676
affect               0.662
unmistakable         0.66
benefit              0.659
adverse_effect       0.653
result               0.645
personal_observation 0.645


In [176]:
efficacy += get_related_terms('effects', topn=3)

side_affect          0.679
side_effect          0.647
effects!.            0.592


In [177]:
efficacy += get_related_terms('notice', topn=9)

noticeable           0.805
noticable            0.769
noticible            0.767
noticed              0.737
improvement!.        0.705
uplifted_mood        0.7
weeksi               0.689
noticiable           0.688
feel                 0.677


In [178]:
efficacy += get_related_terms('result', topn=4)

results              0.713
results!.            0.706
outcome              0.684
improvement          0.656


In [179]:
efficacy += get_related_terms('high_quality', topn=3)

quality              0.857
pure_encapsulation   0.718
unsurpassed          0.717


In [180]:
efficacy += ['works', 'work']

In [221]:
efficacy += get_related_terms('immediate', topn=30)

efficacy += get_related_terms('improve', topn=30)

efficacy += get_related_terms('huge_difference', topn=16)

efficacy = [e for e in efficacy if e not in ['wellbe', 'urogenital', 'blepharitis']]

instant              0.694
instantaneous        0.668
gradual              0.666
dramatic             0.643
profound             0.642
marked               0.637
noticeable           0.63
cumulative           0.628
remarkable           0.625
substantial          0.619
significant          0.614
positive             0.613
notable              0.61
immediately!.        0.604
tremendous           0.604
drastic              0.602
unmistakable         0.601
measurable           0.601
immediately          0.594
tangible             0.591
immediatley          0.589
definate             0.588
appreciable          0.586
imediately           0.585
miraculous           0.584
analgesic            0.583
enegry               0.581
stunned              0.581
energy-              0.581
noticed              0.577
improved             0.815
improved!.           0.692
dramatically         0.69
improvement          0.689
enhance              0.68
significant_improvement 0.674
wellbe               0.67
ove

In [222]:
len(efficacy)

104

In [182]:
cost = get_related_terms('cost', topn=14)

expensive            0.762
pay                  0.756
price                0.725
cheap                0.722
dollar               0.707
costly               0.696
fraction             0.667
markup               0.66
monie                0.658
rite_aid             0.653
costing              0.652
costwise             0.643
expense              0.642
money                0.642


In [183]:
cost.remove('rite_aid')

In [184]:
cost += get_related_terms('price', topn=27)

pricing              0.821
price-               0.775
prices!.             0.764
selling              0.757
reasonable_price     0.742
price!.              0.739
quantity             0.731
quanity              0.731
cost!.               0.725
cost                 0.725
value                0.723
price!!!.            0.72
value-               0.718
priced!.             0.714
brick_and_mortar     0.711
pricei               0.711
price!i              0.706
bargain              0.702
bargin               0.701
product/             0.699
wholesale_club       0.698
affordability        0.697
prices               0.696
quality!!.           0.695
potency!.            0.694
savings!.            0.692
affordable           0.688


In [185]:
cost.remove('brick_and_mortar')

In [186]:
cost += get_related_terms('money', topn=20)

monie                0.78
cash                 0.762
money!.              0.746
dollar               0.732
buck                 0.725
buying               0.708
cost!.               0.68
expensive            0.67
costing              0.665
save                 0.662
spend                0.654
penny                0.653
wisely               0.644
savings!.            0.643
cost                 0.642
expense              0.633
selling              0.633
convienence          0.632
pay                  0.629
price                0.626


In [187]:
cost += get_related_terms('expensive', topn=22)

pricey               0.823
pricy                0.812
cheap                0.809
costly               0.788
cost                 0.762
pay                  0.694
spendy               0.677
overpriced           0.673
money                0.67
expensive-           0.655
skinceuticals        0.651
rite_aid             0.646
inexpensive          0.645
monie                0.642
cheaper              0.639
opinon               0.635
prohibitively        0.633
affordable           0.633
dollar               0.631
compare              0.628
exorbitantly         0.623
savings!.            0.621


In [188]:
cost.remove('skinceuticals')
cost.remove('opinon')
cost.remove('rite_aid')

In [189]:
cost += ['substantial_saving', 'overpay', 'pay', 'thrifty', 'deal', 'costing', 'worth_every_penny',
        'worthwhile_investment', 'worthwhile', 'investment', 'pricey', 'competitive_pricing']

In [190]:
len(cost)

90

In [191]:
service = get_related_terms('seller', topn=27)

vendor               0.886
merchandise          0.777
resend               0.775
sevice               0.773
promply              0.772
sellers              0.77
customer_service     0.762
service              0.759
promptness           0.758
courteous            0.756
prompt               0.752
delievery            0.751
a+++++++             0.748
merchant             0.747
shipment             0.742
lightn               0.74
product/             0.74
discrib              0.737
packageing           0.732
nutricity            0.732
sender               0.732
vender               0.731
supplier             0.73
company              0.728
a++++++++++          0.727
shipping!!.          0.726
fulfillment          0.725


In [192]:
service += get_related_terms('shipping', topn=60)

ship                 0.834
shipment             0.812
prime_member         0.78
packageing           0.771
delivery             0.77
super_saver          0.763
lightn               0.757
handling             0.755
sevice               0.755
delievery            0.753
prime_shipping       0.735
free_shipping        0.734
effecient            0.727
fulfillment          0.724
seller               0.714
delivery!.           0.713
product/             0.711
supersaver           0.71
merchandise          0.709
prime_membership     0.709
vendor               0.708
eligible             0.706
shipper              0.703
shipping!.           0.703
shippment            0.698
prime_account        0.697
service              0.694
prime                0.694
packing              0.693
turnaround           0.693
mailing              0.69
msrp                 0.687
courteous            0.686
amazons              0.686
timeliness           0.685
euro                 0.684
sellers              0.683
apo  

In [193]:
service += get_related_terms('service', topn=60)

service!!.           0.856
service!!!.          0.821
product/             0.81
service!.            0.809
sevice               0.806
delievery            0.8
fulfillment          0.792
delivery!.           0.786
prompt_service       0.78
thanks!!.            0.779
shipping!!.          0.777
quick_delivery       0.774
packageing           0.774
proudct              0.774
reliable!.           0.769
promptness           0.768
merchandise          0.765
lightn               0.765
a+++++++             0.763
buyherbs             0.763
prompt               0.763
superfast            0.761
fast_delivery        0.759
merchant             0.759
seller               0.759
values4u             0.756
prompt_delivery      0.756
transaction          0.754
courteous            0.751
vendor!.             0.75
aaa+                 0.75
quality-             0.749
timely_delivery      0.749
shipping!.           0.744
expedient            0.743
shippment            0.743
promply              0.743
timelin

In [194]:
service += ['ship', 'customer_satisfaction', 'kindness', 'promptness', 'lifelong_customer', 'satisfied_customer', 
            'policy', 'business', 'timeliness', 'timely', 'timely_manner', 'superfast', 'expediently', 'packaged',
           'swift', 'timely_arrival', 'return', 'order']

In [195]:
service += get_related_terms('refund', topn=30)

resend               0.764
gracious             0.757
credit_card          0.735
reimburse            0.725
email                0.725
apologize            0.724
e_mail               0.721
invoice              0.711
compensation         0.708
apology              0.703
inquire              0.7
sender               0.7
paypal               0.693
unethical            0.688
costumer             0.687
vitanherbs           0.687
considerate          0.685
send                 0.683
notify               0.678
policy               0.674
backorder            0.671
customer_satisfaction 0.669
phone_call           0.667
representative       0.666
sellers              0.664
honor                0.663
customer_service     0.662
vitaminlife          0.657
reshipp              0.657
oversight            0.654


In [196]:
service += get_related_terms('company', topn=30)

customer_service     0.784
manufacturer         0.767
unsurpassed          0.767
helpfully            0.745
professionalism      0.744
company!.            0.739
customer_satisfaction 0.739
implicitly           0.735
ethic                0.728
seller               0.728
puritans             0.723
naturwise            0.718
reputable_company    0.717
uphold               0.717
costumer_service     0.712
meticulous           0.711
top_notch            0.711
supplier             0.709
exemplary            0.706
manufacturing_practice 0.703
shady                0.703
bulksuppliments      0.702
pure_encapsulation   0.702
stellar_reputation   0.699
bulksupplements.com  0.697
vendor               0.697
considerate          0.695
verification         0.695
sourcing             0.693
kindness             0.693


In [199]:
service = [e for e in service if e not in ['product/', 'lightn', 'discrib', 'nutricity', 'a++++++++++', 'apo', 'aj', 's_h', 'maui', 
                          'vitaminlife', 'a+++++++', 'aaa+', 'buyherbs', 'reciv', 'vitanherbs', 'vitaminlife',
                          'puritans', 'naturwise', 'bulksupplements.com', 'pure_encapsulation']]

In [239]:
vocab_words = []
for k, v in vocab_dictionary.token2id.items():
    vocab_words.append(k)

In [242]:
efficacy = [e for e in efficacy if e in vocab_words]
cost = [e for e in cost if e in vocab_words]
service = [e for e in service if e in vocab_words]

In [243]:
efficacy = list(set(efficacy))

In [244]:
cost = list(set(cost))

In [245]:
service = list(set(service))

In [246]:
print(efficacy)

['quality', 'marked_difference', 'mental_sharpness', 'effects!.', 'difference', 'analgesic', 'differance', 'negative_side_effect', 'drastic', 'noticible', 'instant', 'improvement!.', 'dramatic', 'enhance', 'notable', 'benefit', 'results', 'definate', 'significant', 'significant_improvement', 'adverse_effect', 'overall', 'tangible', 'increases', 'stunned', 'appreciable', 'clearer', 'gradual', 'vast_improvement', 'works', 'tremendous', 'substantial', 'dramatically', 'noticeably', 'miraculous', 'side_affect', 'remarkable', 'drastically', 'affect', 'noticeable', 'increase', 'marked', 'dramatic_improvement', 'outcome', 'definite_improvement', 'immediately!.', 'marked_improvement', 'feel', 'positive_effect', 'measurable', 'result', 'instantaneous', 'improved', 'deference', 'promotes', 'greatly', 'noticable', 'diference', 'noticed', 'difference!.', 'work', 'results!.', 'immediately', 'side_effect', 'improvment', 'big_difference', 'cumulative', 'help', 'noticeable_difference', 'positive', 'imp

In [247]:
print(cost)

['bargin', 'investment', 'compare', 'overpriced', 'spend', 'inexpensive', 'cost!.', 'price-', 'price', 'dollar', 'expensive', 'competitive_pricing', 'selling', 'quanity', 'save', 'monie', 'worth_every_penny', 'pricing', 'bargain', 'product/', 'affordability', 'pricy', 'worthwhile', 'money', 'value', 'buck', 'reasonable_price', 'price!i', 'quantity', 'overpay', 'spendy', 'money!.', 'deal', 'cheap', 'buying', 'wisely', 'penny', 'cost', 'price!!!.', 'pay', 'expense', 'affordable', 'pricey', 'fraction', 'cash', 'price!.', 'prices', 'cheaper', 'costly']


In [248]:
print(service)

['payment', 'quick_delivery', 'shipping!.', 'prime_membership', 'delivery', 'service!!.', 'aaa', 'backorder', 'return', 'supplier', 'fast_shipping', 'packing', 'verification', 'order', 'service', 'merchant', 'transaction', 'seller', 'satisfied_customer', 'courteous', 'business', 'fulfillment', 'amazons', 'vendor', 'inquire', 'delievery', 'shipper', 'superfast', 'timely', 'costumer', 'sender', 'apologize', 'credit_card', 'ship', 'supersaver', 'top_notch', 'promptness', 'prompt', 'vender', 'customer_service!.', 'customer_satisfaction', 'shippment', 'service!.', 'company!.', 'customer_service', 'send', 'prime_member', 'timely_delivery', 'sourcing', 'ordering', 'merchandise', 'company', 'manufacturer', 'free_shipping', 'service!!!.', 'timely_manner', 'ontime', 'email', 'eligible', 'considerate', 'prime_shipping', 'phone_call', 'ethic', 'representative', 'notify', 'invoice', 'delivery!.', 'prompt_service', 'unsurpassed', 'e_mail', 'honor', 'shipment', 'proudct', 'swift', 'speedy_delivery', 

In [260]:
# convert topics' words to ids
efficacy_id = [vocab_dictionary.token2id[word] for word in efficacy]
cost_id = [vocab_dictionary.token2id[word] for word in cost]
service_id = [vocab_dictionary.token2id[word] for word in service]

In [262]:
topics = [efficacy_id, cost_id, service_id]
num_topics = 5

# initialize eta: matrix of shape num_topics x num_words
eta = np.ones((num_topics, len(vocab_dictionary))) * (1.0/num_topics)
baseline_prob = eta[0, 0]

boosted_prob = 0.9

for i in range(len(topics)):
    
    topic_ids = topics[i]   # word ids related to topic i
    
    for seeded_word_id in topics[i]:
        # boost probability of selected word in current topic
        eta[i, seeded_word_id] = boosted_prob
        # distribute the leftover probability over the remaining topics
        leftover_topic_rownums = [x for x in range(num_topics) if x != i]
        eta[leftover_topic_rownums, seeded_word_id] = ((1 - boosted_prob) / (num_topics - 1))

In [264]:
eta[4,0]

0.20000000000000001

In [265]:
eta[0, vocab_dictionary.token2id['significant']]

0.90000000000000002

In [266]:
eta[1, vocab_dictionary.token2id['significant']]

0.024999999999999994

In [267]:
eta[2, vocab_dictionary.token2id['significant']]

0.024999999999999994

In [268]:
max(np.sum(eta, axis=0))   # sanity check

1.0

In [269]:
eta.shape

(5, 13850)

In [271]:
%%time

# Set training parameters.
num_topics_list = [5]
chunksize = 7000    # number of docs processed at a time
passes = 4
iterations = 400
eval_every = None    # change to 1 if you want to check for convergence
eta = eta           # asymmetric prior (seeded)
alpha = 'auto'       # asymmetric prior learned from data

coherenceList_umass = []

for num_topics in tqdm(num_topics_list):
    lda_seeded = LdaModel(bow_corpus, num_topics=num_topics, id2word=vocab_dictionary, chunksize=chunksize, 
                          passes=passes, eta=eta, alpha=alpha, eval_every=eval_every, iterations=iterations, 
                          random_state=42)
    save_df_s3(lda_seeded, bucket_name, filepath='amazon_reviews/kk/lda_seeded_{}.pkl'.format(num_topics), 
               filetype='pickle')
    print('Trained LDA for {} topics...'.format(num_topics))
    
    # performance metric
    cm = CoherenceModel(model=lda_seeded, corpus=bow_corpus, dictionary=vocab_dictionary, coherence='u_mass')
    coherenceList_umass.append(cm.get_coherence())
    print('Calculated coherence score...: ', cm.get_coherence())
    
    # visualization
    vis = pyLDAvis.gensim.prepare(lda_seeded, bow_corpus, vocab_dictionary)
    plot_fname = 'pyLDAvis_{}.html'.format(num_topics)
    pyLDAvis.save_html(vis, plot_fname)
    print('Saved html file: ', plot_fname)

  0%|          | 0/1 [00:00<?, ?it/s]INFO : using autotuned alpha, starting with [0.2, 0.2, 0.2, 0.2, 0.2]
INFO : using serial LDA version on this node
INFO : running online (multi-pass) LDA training, 5 topics, 4 passes over the supplied corpus of 217530 documents, updating model once every 7000 documents, evaluating perplexity every 0 documents, iterating 400x with a convergence threshold of 0.001000
INFO : PROGRESS: pass 0, at document #7000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6772/7000 documents converged within 400 iterations
INFO : optimized alpha [0.077892676, 0.082816936, 0.10946905, 0.12279554, 0.10918184]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.078): 0.022*"pedometer" + 0.019*"day" + 0.018*"step" + 0.017*"like" + 0.016*"good" + 0.016*"great" + 0.016*"work" + 0.014*"use" + 0.010*"product" + 0.010*"walk"
INFO : topic #1 (0.083): 0.026*"pedometer" + 0.022*"great" + 

DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.11758024, 0.14169693, 0.12300795, 0.092379905, 0.058417365]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.118): 0.017*"help" + 0.015*"day" + 0.015*"work" + 0.014*"good" + 0.012*"product" + 0.012*"feel" + 0.010*"try" + 0.010*"like" + 0.010*"use" + 0.009*"supplement"
INFO : topic #1 (0.142): 0.031*"product" + 0.030*"good" + 0.026*"great" + 0.018*"taste" + 0.015*"use" + 0.012*"like" + 0.011*"price" + 0.010*"work" + 0.010*"supplement" + 0.009*"find"
INFO : topic #2 (0.123): 0.048*"product" + 0.030*"use" + 0.025*"good" + 0.024*"great" + 0.016*"work" + 0.016*"skin" + 0.012*"love" + 0.012*"year" + 0.011*"recommend" + 0.011*"oil"
INFO : topic #3 (0.092): 0.030*"product" + 0.021*"use" + 0.020*"day" + 0.019*"work" + 0.015*"good" + 0.014*"pain" + 0.013*"great" + 0.012*"time" + 0.011*

INFO : topic #1 (0.181): 0.037*"good" + 0.031*"product" + 0.025*"great" + 0.018*"taste" + 0.016*"use" + 0.013*"price" + 0.013*"like" + 0.011*"supplement" + 0.010*"find" + 0.010*"brand"
INFO : topic #2 (0.136): 0.058*"product" + 0.033*"use" + 0.030*"great" + 0.025*"good" + 0.017*"work" + 0.015*"skin" + 0.013*"love" + 0.012*"recommend" + 0.011*"year" + 0.010*"help"
INFO : topic #3 (0.101): 0.027*"product" + 0.021*"day" + 0.021*"work" + 0.019*"pain" + 0.019*"use" + 0.012*"help" + 0.012*"good" + 0.011*"year" + 0.011*"time" + 0.010*"try"
INFO : topic #4 (0.056): 0.048*"hair" + 0.019*"use" + 0.018*"nail" + 0.016*"grow" + 0.011*"product" + 0.011*"eye" + 0.009*"work" + 0.008*"notice" + 0.008*"love" + 0.008*"month"
INFO : topic diff=0.249931, rho=0.277350
INFO : PROGRESS: pass 0, at document #98000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 7000/7000 documents converged within 400 iterations
INFO : optimized alpha [0.16080891, 0.18968856, 0.13901208, 0.10165233, 0.

INFO : topic #4 (0.057): 0.050*"hair" + 0.021*"nail" + 0.018*"grow" + 0.016*"use" + 0.015*"eye" + 0.010*"notice" + 0.010*"coconut_oil" + 0.009*"product" + 0.008*"month" + 0.008*"strong"
INFO : topic diff=0.179453, rho=0.229416
INFO : PROGRESS: pass 0, at document #140000/217530
DEBUG : performing inference on a chunk of 7000 documents
DEBUG : 6999/7000 documents converged within 400 iterations
INFO : optimized alpha [0.18053673, 0.24050231, 0.16356511, 0.10862023, 0.057097871]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.181): 0.018*"help" + 0.016*"day" + 0.015*"work" + 0.012*"good" + 0.012*"feel" + 0.011*"supplement" + 0.011*"product" + 0.010*"try" + 0.009*"year" + 0.009*"recommend"
INFO : topic #1 (0.241): 0.038*"good" + 0.028*"product" + 0.026*"great" + 0.024*"taste" + 0.015*"like" + 0.014*"price" + 0.013*"use" + 0.012*"vitamin" + 0.011*"easy" + 0.010*"brand"
INFO : topic #2 (0.164): 0.064*"product" + 0.034*"g

INFO : optimized alpha [0.19820184, 0.26557565, 0.19120696, 0.11822807, 0.060456198]
DEBUG : updating topics
INFO : merging changes from 7000 documents into a model of 217530 documents
INFO : topic #0 (0.198): 0.018*"help" + 0.015*"day" + 0.014*"work" + 0.012*"supplement" + 0.012*"good" + 0.011*"feel" + 0.011*"product" + 0.010*"try" + 0.009*"recommend" + 0.008*"year"
INFO : topic #1 (0.266): 0.037*"good" + 0.027*"product" + 0.026*"great" + 0.023*"taste" + 0.016*"like" + 0.013*"vitamin" + 0.013*"easy" + 0.012*"price" + 0.012*"use" + 0.011*"supplement"
INFO : topic #2 (0.191): 0.072*"product" + 0.036*"great" + 0.033*"use" + 0.027*"good" + 0.016*"work" + 0.015*"love" + 0.014*"recommend" + 0.013*"skin" + 0.013*"oil" + 0.010*"year"
INFO : topic #3 (0.118): 0.025*"work" + 0.022*"day" + 0.021*"product" + 0.017*"use" + 0.015*"pain" + 0.013*"week" + 0.013*"try" + 0.012*"feel" + 0.012*"help" + 0.011*"start"
INFO : topic #4 (0.060): 0.060*"hair" + 0.029*"nail" + 0.024*"grow" + 0.015*"use" + 0.013

...

INFO : CorpusAccumulator accumulated stats from 14000 documents
INFO : CorpusAccumulator accumulated stats from 15000 documents
INFO : CorpusAccumulator accumulated stats from 16000 documents
INFO : CorpusAccumulator accumulated stats from 17000 documents
INFO : CorpusAccumulator accumulated stats from 18000 documents
INFO : CorpusAccumulator accumulated stats from 19000 documents
INFO : CorpusAccumulator accumulated stats from 20000 documents
INFO : CorpusAccumulator accumulated stats from 21000 documents
INFO : CorpusAccumulator accumulated stats from 22000 documents
INFO : CorpusAccumulator accumulated stats from 23000 documents
INFO : CorpusAccumulator accumulated stats from 24000 documents
INFO : CorpusAccumulator accumulated stats from 25000 documents
INFO : CorpusAccumulator accumulated stats from 26000 documents
INFO : CorpusAccumulator accumulated stats from 27000 documents
INFO : CorpusAccumulator accumulated stats from 28000 documents
INFO : CorpusAccumulator accumulated sta

INFO : CorpusAccumulator accumulated stats from 142000 documents
INFO : CorpusAccumulator accumulated stats from 143000 documents
INFO : CorpusAccumulator accumulated stats from 144000 documents
INFO : CorpusAccumulator accumulated stats from 145000 documents
INFO : CorpusAccumulator accumulated stats from 146000 documents
INFO : CorpusAccumulator accumulated stats from 147000 documents
INFO : CorpusAccumulator accumulated stats from 148000 documents
INFO : CorpusAccumulator accumulated stats from 149000 documents
INFO : CorpusAccumulator accumulated stats from 150000 documents
INFO : CorpusAccumulator accumulated stats from 151000 documents
INFO : CorpusAccumulator accumulated stats from 152000 documents
INFO : CorpusAccumulator accumulated stats from 153000 documents
INFO : CorpusAccumulator accumulated stats from 154000 documents
INFO : CorpusAccumulator accumulated stats from 155000 documents
INFO : CorpusAccumulator accumulated stats from 156000 documents
INFO : CorpusAccumulator 

Calculated coherence score...:  -2.38490514601


DEBUG : performing inference on a chunk of 217530 documents
DEBUG : 217530/217530 documents converged within 400 iterations
/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
100%|██████████| 1/1 [11:43<00:00, 703.43s/it]

Saved html file:  pyLDAvis_5.html
CPU times: user 11min 43s, sys: 1.9 s, total: 11min 45s
Wall time: 11min 43s


In [ ]:
    save_df_s3(lda_seeded, bucket_name, filepath='amazon_reviews/kk/lda_seeded_{}.pkl'.format(num_topics), 
               filetype='pickle')

In [6]:
lda_temp = load_df_s3(bucket_name, filepath='amazon_reviews/kk/lda_seeded_3.pkl', filetype='pickle')